In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml

In [62]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [203]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
data_generator.fraud.time
importlib.reload(data_generator.fraud.time)
# importlib.invalidate_caches()
from data_generator.general_time import *
from data_generator.utils import build_transaction, ConfigForTrans
from data_generator.fraud.txndata import FraudTransPartialData, TransAmount, sample_category
from data_generator.fraud.time import derive_from_last_time

In [4]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [5]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [6]:
os.chdir("..")

In [12]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [39]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)

In [44]:
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)

In [41]:
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик

In [88]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0]
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")

# Генерация времени

**Датафрейм под транзакции. Этап тестов**<br>
Создадим пустой датафрейм под транзакции. Будем копировать и использовать его при тестах функций.

In [24]:
transactions = pd.DataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "txn_time": pd.Series(dtype="datetime64[ns]"),
             "unix_time":pd.Series(dtype="int64"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="bool"),
            "merchant_id":pd.Series(dtype="int64"),
             "trans_city":pd.Series(dtype="string"),
            "trans_lat":pd.Series(dtype="float64"),
             "trans_lon":pd.Series(dtype="float64"),
            "trans_ip":pd.Series(dtype="string"),
             "device_id":pd.Series(dtype="int64"),
             "account": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="bool"),
             "is_suspicious": pd.Series(dtype="bool"),
            "status":pd.Series(dtype="string"),
            "rule":pd.Series(dtype="string")})
transactions

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule


**Функция получения случайных значений из `truncnorm` распределения**
- пойдет в модуль `utils`

In [124]:
def get_values_from_truncnorm(low_bound, high_bound, mean, std, size=1):
    """
    Сгенерировать массив чисел из обрезанного нормального распределения.
    Можно сгенерировать массив с одним числом
    ------------
    low_bound - float, int. Нижняя граница значений
    high_bound - float, int. Верхняя граница значений 
    mean - float, int. Среднее
    std - float, int. Стандартное отклонение
    size - Количество чисел в возвращаемом массиве
    ------------
    Возвращает np.ndarray
    """
    return truncnorm.rvs((low_bound - mean) / std, (high_bound - mean) / std, loc=mean, scale=std, size=size)

## Подфункция генерации времени c прибавлением к времени последней транзакции `derive_from_last_time`

In [211]:
derive_last = pd_timestamp_to_unix(pd.to_datetime("2025-06-20 11:57:12", format="%Y-%m-%d %H:%M:%S"))
derive_last
# pd.to_datetime(derive_last, unit="s")

1750420632

In [214]:
derive_from_last_time(last_txn_unix=derive_last, lag_interval=0, min=30, max=60, random_lag=True)

(Timestamp('2025-06-20 12:43:12'), 1750423392)

## Класс `DropConfigs`
- конфигурации на основании которых будут генерироваться транзакции дропов

In [282]:
@dataclass
class DropConfigs:
    """
    Это данные на основе которых будут генерироваться транзакции
    ---------------------
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int. Количество входящих транзакций после которых дроп уходит на паузу.
    period_out_lim: int. Количество исходящих транзакций после которых дроп уходит на паузу.
    lag_interval: int. Минуты. На сколько дроп должен делать паузу после
                       достижения лимита входящих и/или исходящих транзакций.
                       Например 1440 минут(сутки). Отсчет идет от первой транзакции в последнем периоде активности.
    two_way_delta: dict. Минимум и максимум дельты времени. Для случаев когда дельта может быть и положительной и отрицательной.
                         Эта дельта прибавляется к lag_interval, чтобы рандомизировать время возобновления активности,
                         чтобы оно не было ровным. Берется из конфига drops.yaml
    pos_delta: dict. Минимум и максимум случайной дельты времени в минутах. Для случаев когда дельта может быть только положительной.
                          Эта дельта - промежуток между транзакциями дропа в одном периоде. Просто прибавляется ко времени последней транзакции.
    """
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int
    period_out_lim: int
    lag_interval: int
    two_way_delta: dict
    pos_delta: dict

### Создание аргументов для `DropConfigs`

In [283]:
# timestamp-ы использованные еще для генерации легальных и purchase фрод транзакций
timestamps = pd.read_parquet("./data/generated_data/timestamps.parquet", engine="pyarrow")

In [284]:
time_weight_args = time_cfg["time_weights_args"]
time_weight_args

{'Offline_24h_Legit': {'is_fraud': False,
  'round_clock': True,
  'online': False},
 'Offline_24h_Fraud': {'is_fraud': True, 'round_clock': True, 'online': False},
 'Online_Legit': {'is_fraud': False, 'round_clock': True, 'online': True},
 'Online_Fraud': {'is_fraud': True, 'round_clock': True, 'online': True},
 'Offline_Day_Legit': {'is_fraud': False,
  'round_clock': False,
  'online': False},
 'Offline_Day_Fraud': {'is_fraud': True,
  'round_clock': False,
  'online': False}}

In [285]:
time_weights_dict = get_all_time_patterns(time_weight_args)

**Легальные транзакции**  
Нужно знать их количество для определения числа фрод транзакций  
и следовательно, количества клиентов для их генерации

In [34]:
legit_txns = pd.read_parquet("./data/generated_data/legit_trans_148K.parquet", engine="pyarrow")

**Клиенты для дроп фрода**  
Случайно возьмем клиентов, у которых нет никаких транзакций  
Для этого загрузим семплированных для легальных транзакций клиентов, т.е. у кого были созданы легальные транзакции.  
У части этих клиентов также созданы purchase фрод транзакции. Всех этих клиентов надо исключить.

In [29]:
clients_sample_df = gpd.read_file("./data/cleaned_data/clients_sample.gpkg")

In [30]:
clients_sample_df.shape

(3000, 12)

Конфиги загружаются временно тут

In [286]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [287]:
# доля дроп фрода - Дропы занимающиеся переводами и снятиями; и дропы, которые покупают товары на присланные деньги
fraud_rate = fraud_cfg["fraud_rate"]
drop_flow_share = drops_cfg["drop_distributor"]["rate"] # доля от всего фрода
drop_purchase_share = drops_cfg["drop_purchaser"]["rate"] # доля от всего фрода

In [288]:
# лимит на исходящие транзакции дропа. Нужен сейчас для вычисления количества клиентов дропов

drop_out_lim = drops_cfg["drop_distributor"]["out_lim"]

In [289]:
# отсюда посчитаем количество клиентов для дроп фрода

legit_count = legit_txns.shape[0]
drop_fraud_count = round(legit_count * fraud_rate *(drop_flow_share + drop_purchase_share))
drops_count = round(drop_fraud_count / drop_out_lim)
drops_count

56

In [55]:
# Клиенты незадействованные ранее

not_used_clients = clients_with_geo.loc[~clients_with_geo.client_id.isin(clients_sample_df.client_id)].copy()

In [56]:
# семплируем drops_count клиентов

clients_for_drop = not_used_clients.sample(n=drops_count).reset_index(drop=True)

In [57]:
clients_for_drop.head(2)

,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,966,13,1978-09-02,male,Краснодарский,Сочи,UTC+3,43.585583,39.723142,343285,2.60.3.149,"MULTIPOLYGON (((39.14937 44.02339, 39.14982 44..."
1,12755,55,1959-11-01,female,Самарская,Тольятти,UTC+4,53.520644,49.389461,719484,2.60.20.162,"MULTIPOLYGON (((49.14714 53.55389, 49.15065 53..."


In [58]:
clients_for_drop.shape

(56, 12)

## Генерация счетов клиентов и внешних счетов

**Счета клиентов**  

Пускай начинаются с 10000

In [462]:
accounts = clients_with_geo[["client_id"]].copy()

In [463]:
accounts["account_id"] = 1

In [464]:
accounts.loc[0, "account_id"] = 10000

In [465]:
accounts.head()

,client_id,account_id
0,1,10000
1,2,1
2,3,1
3,4,1
4,5,1


In [466]:
# Кумулятивно складываем числа в серии. Получается в каждой записи будет результат сложения текущего и всех предыдущих чисел
# Т.е. 10000, 10000 + 1, 10001 + 1 и т.д. Так будут счета с номерами от 10000 до 10000 + n-1 клиентов

accounts["account_id"] = accounts["account_id"].cumsum()

In [467]:
accounts.head()

,client_id,account_id
0,1,10000
1,2,10001
2,3,10002
3,4,10003
4,5,10004


In [468]:
accounts.agg({"account_id":["min","max"]})

,account_id
min,10000
max,15368


In [469]:
assert accounts.shape[0] == accounts.account_id.nunique(), "Values in account_id are not unique!"

In [470]:
accounts.shape[0]

5369

In [534]:
# Колонка is_drop. Дроп клиент или нет. Пока нет дропов.
# Они будут обозначаться непосредственно во время генерации активности дропов

accounts["is_drop"] = False

In [535]:
accounts.head()

,client_id,account_id,is_drop
0,1,10000,False
1,2,10001,False
2,3,10002,False
3,4,10003,False
4,5,10004,False


**Внешние счета**  
Счета начинающиеся с максимального номера счета нашего клиента + 1

In [473]:
# Пусть будет 10000 счетов

start_id = accounts.account_id.max() + 1
outer_accounts = pd.Series(data=np.arange(start_id, start_id + 10000, step=1), name="account_id", dtype="int")

In [476]:
outer_accounts.iloc[np.r_[0:3,-3:0]]

0       15369
1       15370
2       15371
9997    25366
9998    25367
9999    25368
Name: account_id, dtype: int64

In [477]:
# Не должно быть пересечений по account_id

assert accounts.merge(outer_accounts, on="account_id").empty, "Clients account ids are in the outer account ids"

**Сохранение счетов в файлы**

In [479]:
accounts.to_csv("./data/generated_data/accounts.csv", index=False)

In [480]:
outer_accounts.to_csv("./data/generated_data/outer_accounts.csv", index=False)

## Конфиги для дроп фрода, объект датакласса `DropConfigs`

In [481]:
# Импорт на время разработки

importlib.reload(data_generator.utils)
from data_generator.utils import DropConfigs

**Доп. конфиги из `drop.yaml`**

In [374]:
period_in_lim = drops_cfg["drop_distributor"]["period_in_lim"]
period_out_lim = drops_cfg["drop_distributor"]["period_out_lim"]
lag_interval = drops_cfg["lag_interval"]
two_way_delta = drops_cfg["two_way_delta"]
pos_delta = drops_cfg["pos_delta"]

In [536]:
drop_configs = DropConfigs(clients=clients_for_drop, timestamps=timestamps, transactions=None, 
                           accounts=accounts, outer_accounts=outer_accounts,
                           client_devices=client_devices, offline_merchants=offline_merchants, 
                           categories=cat_stats_full, online_merchant_ids=online_merchant_ids, \
                           time_weights_dict=time_weights_dict, rules=None, cities=districts_ru, \
                           fraud_amounts=None, period_in_lim=period_in_lim, period_out_lim=period_out_lim, \
                           lag_interval=lag_interval, two_way_delta=two_way_delta, pos_delta=pos_delta
                              )

In [84]:
sample_category(fraud_configs.categories, online=True, is_fraud=True, rule="blalb")

,category,avg_amt,amt_std,cat_count,online,share,fraud_count,fraud_share,round_clock
5,shopping_net,1252.224798,3558.296372,41779,True,0.07518,506,0.012111,True


## Объект класса `FraudTransPartialData`
- генерация части данных транзакции

In [114]:
trans_part_test = FraudTransPartialData(merchants_df=offline_merchants, client_info=None, \
                                        online_merchant_ids=online_merchant_ids, fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)

In [115]:
for client_info in clients_with_geo.loc[[1]].itertuples():
    trans_part_test.client_info = client_info
trans_part_test.client_info

Pandas(Index=1, client_id=2, district_id=1, birth_date='1945-02-04', sex='male', region='Москва', area='Москва', timezone='UTC+3', lat=55.7538789, lon=37.6203735, population=11514330, home_ip='2.60.0.2', geometry=<MULTIPOLYGON (((37.291 55.802, 37.295 55.803, 37.297 55.803, 37.298 55.803,...>)

In [116]:
trans_part_test.original_data(online=True, receive=True)

(nan,
 nan,
 nan,
 'not applicable',
 'not applicable',
 <NA>,
 'transfer',
 'inbound')

In [117]:
trans_part_test.original_data(online=True, receive=False)

(nan,
 55.7538789,
 37.6203735,
 '2.60.0.2',
 'Москва',
 np.int64(3),
 'transfer',
 'outbound')

In [118]:
trans_part_test.original_data(online=False)

(nan,
 55.7538789,
 37.6203735,
 'not applicable',
 'Москва',
 <NA>,
 'ATM',
 'withdrawal')

In [86]:
trans_part_test.used_ips

Series([], dtype: object)

In [87]:
trans_part_test.used_devices

Series([], dtype: object)

## Объект класса `TransAmount`

In [90]:
fraud_amts_test = TransAmount(categories_stats=cat_fraud_amts)

In [91]:
fraud_amts_test.fraud_amount(category_name="shopping_net")

np.float64(30930.0)

In [92]:
fraud_amts_test.freq_trans_amount(low=3000)

np.float64(3090.0)

## Класс `DropAccountHandler`

In [484]:
class DropAccountHandler:
    """
    Генератор номеров счетов входящих/исходящих транзакций.
    Учет использованных счетов.
    """
    def __init__(self, configs: DropConfigs, client_id, used_accounts=pd.Series(name="account_id")):
        """
        configs - pd.DataFrame. Данные для создания транзакций: номера счетов клиентов и внешних счетов, и др.
        client_id - int. id текущего дропа.
        used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
        """
        self.accounts = configs.accounts
        self.outer_accounts = configs.outer_accounts
        self.client_id = client_id
        # self.account = 0
        self.account = configs.accounts.loc[configs.accounts.client_id == client_id, "account_id"].iat[0]
        self.used_accounts = used_accounts

    def get_account(self, to_drop):
        """
        Номер счета входящего/исходящего перевода
        to_drop - bool. Перевод другому дропу в нашем банке или нет.
        """
        # Если отправляем/получаем из другого банка.  
        if not to_drop:
            # Семплируем номер внешнего счета который еще не использовался
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            # Добавляем этот счет в использованные как последнюю запись в серии
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account
        
        # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
        # Фильтруем accounts исключая свой счет и использованные счета, и выбирая дропов. Для случая если to_drop
        drop_accounts = self.accounts.loc[(self.accounts.client_id != self.client_id) & (self.accounts.is_drop == True) \
                                          & ~(self.accounts.account_id.isin(self.used_accounts))]
        
        # Если счетов дропов ещё нет. Берем внешний неиспользованный счет
        if drop_accounts.empty:
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account

        # Дропы есть
        account = drop_accounts.account_id.sample(1).iat[0]
        # Добавляем этот счет в использованные как последнюю запись в серии
        self.used_accounts.loc[self.used_accounts.shape[0]] = account
        return account


    def reset_cache(self):
        """
        Сброс кэшированных значений.
        Это только использованные номера счетов
        """
        self.used_accounts = pd.Series(name="account_id")

**Тест `DropAccountHandler`**

In [537]:
# импорт на время разработки

import data_generator.fraud.drops.base
importlib.reload(data_generator.fraud.drops.base)
from data_generator.fraud.drops.base import DropAccountHandler

In [554]:
drop_acc_hand1 = DropAccountHandler(configs=drop_configs)

In [556]:
drop_acc_hand1.client_id = 1

In [557]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.account

np.int64(10000)

In [520]:
drop_acc_hand1.get_account()

np.int64(18331)

In [521]:
drop_acc_hand1.used_accounts

0    18331
Name: account_id, dtype: int64

In [541]:
drop_acc_hand1.label_drop()

In [544]:
drop_acc_hand1.accounts.head(2)

,client_id,account_id,is_drop
0,1,10000,True
1,2,10001,False


In [545]:
drop_acc_hand1.client_id = 99

In [547]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.label_drop()

In [549]:
drop_acc_hand1.accounts.query("client_id == 99")

,client_id,account_id,is_drop
93,99,10093,True


In [550]:
drop_acc_hand1.get_account(to_drop=True)

np.int64(10000)

In [551]:
drop_acc_hand1.used_accounts

0    18287
1    10000
Name: account_id, dtype: int64

In [553]:
drop_acc_hand1.reset_cache()
drop_acc_hand1.used_accounts

Series([], Name: account_id, dtype: object)

In [558]:
nodrops_acc = drop_acc_hand1.get_account(to_drop=True)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.account_id == nodrops_acc]

,client_id,account_id,is_drop


In [559]:
drop_acc_hand1.used_accounts.loc[drop_acc_hand1.used_accounts == nodrops_acc]

0    23419
Name: account_id, dtype: int64

## Класс `DropClient`
- метод get_account(self): Мб добавить вариант to_drop: bool. Перевод другому имеющемуся дропу. при этом. 
    - записываем счет дропа в свой счет. Атрибут account
    - записываем  счет дропа в атрибут drop_accounts
    - если надо перевести другому дропу, то фильтруем drop_accounts исключая свой
    - надо тогда клиентам приписать шестизначные счета. просто по порядку.
    - счета куда отправляют дропы это номера начинающиеся со счета последнего клиента + 1

In [259]:
class DropClient: 
    """
    Генератор сумм входящих/исходящих транзакций, сумм снятий, номеров счетов.
    Управление балансом текущего дропа.
    """
    def __init__(self, accounts, account, outer_accounts, balance=0, batch_txns=0, chunk_size=0, used_accounts=pd.Series(name="account_id")):
        """
        accounts - pd.DataFrame. Счета клиентов банка.
        account - int. Номер счета текущего дропа.
        outer_accounts - pd.Series. Номера счетов для входящих и исходящих переводов в/из других банков.
        balance - float. Текущий баланс дропа
        batch_txns - int. Счетчик транзакций сделанных в рамках распределения полученной партии денег
        chunk_size - int, float. Последний созданный размер части баланса для перевода по частям.
        used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
        """
        self.accounts = accounts
        self.account = account
        self.outer_accounts = outer_accounts
        self.balance = balance
        self.batch_txns = batch_txns
        self.chunk_size = chunk_size
        self.used_accounts = used_accounts

    def update_balance(self, amount, add=False, declined=False):
        """
        Увеличить/уменьшить баланс на указанную сумму
        -------------------
        amount - float, int.
        add - bool. Прибавлять сумму или отнимать.
        declined - bool. Отклонена ли транзакция или одобрена.
        """
        # Не обновлять баланс если транзакция отклонена.
        if declined:
            return
            
        # Увеличить баланс   
        if add:
            self.balance += amount
            return
            
        # Уменьшить баланс    
        self.balance -= amount

    def receive(self, declined, low=5000, high=100000, mean=30000, std=20000, round=500):
        """
        Генерация суммы входящего перевода
        --------------------------
        declined - bool. Отклонена ли транзакция или одобрена
        low - float. Минимальная сумма
        high - float. Максимальная сумма
        mean - float. Средняя сумма
        std - float. Стандартное отклонение
        round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце
                     При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1
        """

        # Генерация суммы. Округление целой части при необходимости
        amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // round * round
        
        # Обновляем баланс если транзакция не отклонена
        self.update_balance(amount=amount, add=True, declined=declined)
        
        return amount

    def get_chunk_size(self, online=False, atm_min=10000, atm_share=0.5, round=500, rand_rate=0.9, start=0, stop=0, step=0):
        """
        Вернуть случайный размер суммы перевода для перевода по частям
        либо вернуть долю от баланса для снятия/перевода по частям.
        -------------------------------
        online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля atm_share от баланса, но не меньше atm_min
        atm_min - int, float. Минимальная сумма снятия дропом в банкомате.
        atm_share - float. Доля от баланса если снятие через банкомат.
        round - int. Округление целой части. По умолчанию 500. 
                     Значит суммы будут округлены до тысяч или пяти сотен
        rand_rate - float. От 0 до 1. Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
                           Но возможны случайные совпадаения с предыдущим размером чанка.
                           Доля случайных размеров подряд будет:
                           p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
                           p(rr) - rand_rate
                           p(rr) - (p(rr) * p(c)). Например p(rr) = 0.9, 5 вариантов размеров чанка - p(c) = 0.20
                           0.9 - (0.9 * 0.2) = 0.72
                           В около 72% случаев размеры чанков не будут подряд одинаковыми 
        start - int. Минимальный размер. Прописываем если генерация не через share.
                     То же самое для stop и step
        stop - int. Максимальный размер - не входит в возможный выбор.
                    Максимальное генерируемое значение равно stop - step
        step - int. Шаг размеров.
        --------------------
        Возвращает np.int64
        Результат кэшируется в self.chunk_size
        """
        # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
        # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
        if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
            return self.chunk_size

        # Если перевод
        if online:
            sampling_array = np.arange(start, stop, step)
            self.chunk_size = np.random.choice(sampling_array)
            return self.chunk_size
            
        # Если снятие    
        self.chunk_size = max(atm_min, self.balance * atm_share // round * round)
        return self.chunk_size
            
        
    def one_operation(self, amount=0, declined=False, in_chunks=False):
        """
        Генерация суммы операции дропа.
        ---------
        amount - float, int. Сумма перевода если перевод по частям - in_chunks == True
        declined - bool. Отклонена ли транзакция или одобрена
        in_chunks - bool. Перевод по частям или целиком. Если False, то просто пробуем перевести все с баланса
                          При True нужно указать amount.
        """
        if in_chunks and amount <= 0:
            raise ValueError(f"""If in_chunks is True, then amount must be greater than 0.
Passed amount: {amount}""")

        # Если перевод не по частям. Пробуем перевестит все с баланса. 
        if not in_chunks:
            amount = self.balance
            self.update_balance(amount=self.balance, add=False, declined=declined)
            # Прибавляем счетчик транзакции для текущей партии денег
            self.batch_txns += 1
            return amount

        # Иначе считаем сколько частей исходя из размера одной части
        chunks = self.balance // amount

        # Если целое число частей больше 0. Пробуем перевести одну часть
        if chunks > 0:
            self.update_balance(amount=amount, add=False, declined=declined)
            self.batch_txns += 1
            return amount

        # Если баланс меньше одной части. Пробуем перевести то что осталось
        rest = self.balance
        self.update_balance(amount=rest, add=False, declined=declined)
        self.batch_txns += 1
        return rest

    def get_account(self, to_drop):
        """
        Номер счета входящего/исходящего перевода
        to_drop - bool. Перевод другому дропу в нашем банке или нет.
        """
        # Фильтруем accounts исключая свой счет и выбирая дропов. Для случая если to_drop
        drop_accounts = self.accounts.loc[(self.accounts.account_id != self.account) & (self.accounts.is_drop == True)]

        # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
        if to_drop and not drop_accounts.empty: 
            account = drop_accounts.account_id.sample(1).iat[0]
            # Добавляем этот счет в использованные как последнюю запись в серии
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account

        # Если отправляем/получаем из другого банка.  
        # Семплируем номер внешнего счета который еще не использовался
        account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
        # Добавляем этот счет в использованные как последнюю запись в серии
        self.used_accounts.loc[self.used_accounts.shape[0]] = account
        
        return account

    def reset_cache(self, balance=True, used_accounts=True, chunk_size=True, batch_txns=True):
        """
        Сброс кэшированных значений
        По умолчанию сбрасывается всё. Если что-то надо оставить, то надо выставить False
        для этого
        -----------------
        balance - bool
        used_accounts - bool
        chunk_size - bool
        batch_txns - bool
        """
        if balance:
            self.balance = 0
        if used_accounts:
            self.used_accounts = pd.Series(name="account_id")
        if chunk_size:
            self.chunk_size = 0
        if batch_txns:
            self.batch_txns = 0

In [250]:
drop_client_test = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

In [177]:
drop_client_test.receive(declined=True)

np.float64(40500.0)

In [178]:
drop_client_test.balance

0

In [179]:
drop_client_test.receive(declined=False)

np.float64(29500.0)

In [131]:
drop_client_test.reset_cache()
drop_client_test.update_balance(amount=15000, add=True)
drop_client_test.balance

15000

In [180]:
drop_client_test.batch_txns = 0
# Проверка что размер чанка меняется если batch_txns равно нулю
[drop_client_test.get_chunk_size(online=True, atm_min=10000, atm_share=0.5, round=500, start=5000, stop=25000, step=5000) for _ in range(10)]

[np.int64(15000),
 np.int64(5000),
 np.int64(5000),
 np.int64(10000),
 np.int64(20000),
 np.int64(5000),
 np.int64(5000),
 np.int64(5000),
 np.int64(15000),
 np.int64(15000)]

In [255]:
# Проверка что размер чанка не меняется если batch_txns не равно нулю
drop_client_test.batch_txns = 1
chunks_list = [drop_client_test.get_chunk_size(online=True, atm_min=10000, atm_share=0.5, \
                                                 round=500, rand_rate=0.9, start=5000, stop=30000, step=5000) for _ in range(1000)]
chunks_df = pd.DataFrame(pd.Series(chunks_list, name="chunk_size"))
chunks_df.head()

,chunk_size
0,25000
1,20000
2,20000
3,10000
4,15000


In [256]:
np.arange(start=5000, stop=25000, step=5000)

array([ 5000, 10000, 15000, 20000])

In [257]:
chunks_df["prev_size"] = chunks_df.chunk_size.shift(1)
chunks_df["eq_to_prev"] = chunks_df["chunk_size"] == chunks_df["prev_size"]
chunks_df.head(3)

,chunk_size,prev_size,eq_to_prev
0,25000,NaN,False
1,20000,25000.0,False
2,20000,20000.0,True


In [258]:
chunks_df.eq_to_prev.value_counts(normalize=True)

eq_to_prev
False    0.712
True     0.288
Name: proportion, dtype: float64

In [134]:
drop_client_test.get_account(to_drop=False)

np.int64(24643)

In [135]:
drop_client_test.used_accounts, drop_client_test.chunk_size, drop_client_test.balance

(0    24643
 Name: account_id, dtype: int64,
 np.int64(5000),
 15000)

In [136]:
drop_client_test.reset_cache()

In [137]:
drop_client_test.used_accounts, drop_client_test.chunk_size, drop_client_test.balance

(Series([], Name: account_id, dtype: object), 0, 0)

In [138]:
drop_client_test.update_balance(amount=7500)
drop_client_test.balance

-7500

In [139]:
drop_client_test.update_balance(amount=9900, add=True)
drop_client_test.balance

2400

In [140]:
chunk_size = drop_client_test.get_chunk_size(start=5000, stop=25000, step=5000)
chunk_size

10000

In [1233]:
chunk_size = drop_client_test.get_chunk_size(share=0.25)
chunk_size

np.float64(8000.0)

In [1658]:
drop_client_test.one_operation(amount=chunk_size, declined=False, in_chunks=True)

np.float64(17000.0)

In [1659]:
drop_client_test.balance, drop_client_test.batch_txns

(np.float64(17500.0), 1)

In [1236]:
drop_client_test.one_operation(amount=chunk_size, share=1, declined=True, in_chunks=True)
drop_client_test.balance

np.float64(25000.0)

In [1237]:
drop_client_test.one_operation(amount=0, share=1, declined=True, in_chunks=False)
drop_client_test.balance

np.float64(25000.0)

In [1243]:
drop_client_test.receive(declined=False, low=100000, high=500000, mean=250000, std=150000, round=1000)
drop_client_test.balance

np.float64(152000.0)

In [ ]:
drop_client_test.balance

In [1239]:
drop_client_test.one_operation(amount=0, share=0.5, declined=False, in_chunks=True)
drop_client_test.balance

np.float64(0.0)

In [1153]:
drop_client_test.balance

np.float64(38500.0)

In [1166]:
drop_client_test.transfer(declined=True, in_chunks=False)

np.float64(38500.0)

In [1167]:
drop_client_test.balance

np.float64(38500.0)

In [1242]:
drop_client_test.reset_balance()

np.float64(186491.0)

In [1180]:
drop_client_test.atm(share=0.5, declined=False)

np.float64(93000.0)

In [1181]:
drop_client_test.balance

np.float64(93491.0)

In [1009]:
chunk_size = drop_client_test.get_chunk_size(start=5000, stop=25000, step=5000)
chunk_size

np.int64(15000)

In [1011]:
# drop_client_test.transfer(batch=3, chunk_size=chunk_size)

np.int64(15000)

In [1012]:
drop_client_test.balance

np.float64(43141.0)

In [1013]:
# drop_client_test.transfer(batch=3, chunk_size=chunk_size)
# drop_client_test.transfer(batch=3, chunk_size=chunk_size)
# drop_client_test.transfer(batch=3, chunk_size=chunk_size)

np.float64(13141.0)

In [1014]:
drop_client_test.balance

0

In [1015]:
# drop_client_test.receive(batch=4)

np.float64(26000.0)

In [1016]:
# drop_client_test.reset_balance()
# drop_client_test.balance

0

**тест `get_account`**

In [141]:
test_accounts = accounts.sample(1000).copy()

In [142]:
# Сделаем 10 процентов дропами для тестов

for client in test_accounts.itertuples():
    if np.random.uniform(0, 1) < 0.101:
        test_accounts.loc[client.Index, "is_drop"] = True

In [143]:
test_accounts["is_drop"].value_counts(normalize=True)

is_drop
False    0.909
True     0.091
Name: proportion, dtype: float64

In [144]:
test_acc = test_accounts.account_id.sample(1, random_state=33).iat[0]

drop_cl_test_accs = DropClient(accounts=test_accounts, account=test_acc, outer_accounts=outer_accounts)

In [145]:
test_accounts.account_id.sample(1, random_state=33)

458    10458
Name: account_id, dtype: int64

In [146]:
drop_cl_test_accs.get_account(to_drop=False)

np.int64(22412)

In [147]:
drop_cl_test_accs.used_accounts

0    22412
Name: account_id, dtype: int64

In [148]:
drop_to = drop_cl_test_accs.get_account(to_drop=True)
drop_to

np.int64(11684)

In [149]:
test_accounts.loc[test_accounts.account_id == drop_to]

,client_id,account_id,is_drop
1684,1787,11684,True


In [150]:
drop_cl_test_accs.used_accounts

0    22412
1    11684
Name: account_id, dtype: int64

In [151]:
drop_cl_test_accs.balance

0

## Класс `DropTimeHandler`
- управление временем транзакций дропа

In [460]:
class DropTimeHandler:
    """
    Управление временем транзакций дропа
    """
    def __init__(self, configs: DropConfigs, start_unix=0, last_unix=0, in_lim=2, out_lim=5, in_txns=0, out_txns=0):
        """
        timestamps - pd.DataFrame. Диапазон timestamp-ов с колонками: | timestamp | unix_time |
        in_lim - int. Количество входящих транзакций после которых дроп уходит на паузу
        out_lim - int. Количество исходящих транзакций после которых дроп уходит на паузу
        start_unix - int. Во сколько была первая транзакция в периоде. Нужная для отсчета следующего периода
                          активности. Unix время в секундах. 
        last_unix - int. Время последней транзакции. Unix время в секундах.
        """
        self.configs = configs
        self.start_unix = start_unix
        self.last_unix = last_unix
        self.in_lim = in_lim
        self.out_lim = out_lim
        self.in_txns = in_txns
        self.out_txns = out_txns


    def get_time_delta(self, two_way, minutes=True):
        """
        Получение случайного интервала времени в секундах или минутах из равномерного распределения
        ---------------------
        two_way - bool. Дельта может быть <= 0 и > 0. Если False то только > 0
        minutes - bool. Минуты или секунды
        """
        if two_way:
            delta = np.random.uniform(self.configs.two_way_delta["min"], self.configs.two_way_delta["max"])
        else:
            delta = np.random.uniform(self.configs.pos_delta["min"], self.configs.pos_delta["max"])

        if minutes:
            return round(delta)
            
        return round(delta * 60)
    

    def txns_count(self, receive=False, reset=False):
        """
        Внутренний счетчик входящих и исходящих транзакций за период активности.
        Имеет 4 варианта действий для self.in_txns и self.out_txns: 
        1. сброс счетчиков на in=1, out=0; 
        2. Сброс на in=0, out=1;
        3. in + 1;
        4. out + 1;
        ---------------------
        receive: bool. Входящая транзакция или исходящая.
        reset: bool. Начать отчет заново или нет.
        """
        # Если транзакция входящая. Период начинается заново и с входящей транзакции
        # Эта транзакция будет первой в периоде
        if receive and reset:
            self.in_txns = 1
            self.out_txns = 0
            return
        # Исходящая транзакция и период начинается заново. Эта транзакция будет первой входящей в периоде
        if reset:
            self.in_txns = 0
            self.out_txns = 1
            return
        # Входящая транзакция и период продолжается. Прибавлем счетчик входящих.
        if receive:
            self.in_txns += 1
            return
        # Исходящая транзакция и период продолжается. Прибавлем счетчик исходящих.
        self.out_txns += 1


    def get_txn_time(self, receive, in_txns, lag_interval=1440):
        """
        Генерация времени транзакции
        ------------------
        receive: bool. Текущая транзакция будет входящей или исходящей.
        in_txns: int. Абсолютное количество входящих транзакций на текущий момент, не считая генерируемую.
        lag_interval: int. Желаемый лаг по времени от последней транзакции в минутах.
                            Используется для перерывов в активности дропа. По умолчанию 1440 минут т.е. 24 часа
        """

        # Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции
        if receive and in_txns == 0:
            time_sample = self.configs.timestamps.sample(1)
            txn_time = time_sample.timestamp.iat[0]
            self.last_unix = time_sample.unix_time.iat[0]
            self.start_unix = self.last_unix
            self.in_txns += 1
            return txn_time, self.last_unix

        # Условия для не первых транзакций

        # Если достигнут лимит входящих транзакций для периода активности
        if self.in_txns == self.in_lim:
            # Генерация дельты, чтобы время выглядело не ровным, а случайным.
            # Слагаем её с lag_interval
            time_delta = self.get_time_delta(two_way=True)
            lag_interval = self.configs.lag_interval + time_delta
            # print(time_delta, lag_interval)
            # Сбрасываем счетчик входящих и исходящих транзакций. Отсчет заново для нового периода
            # Входящие = 1, исходящие = 0
            self.txns_count(receive=receive, reset=True)
            # Создаем время и записываем его как время последней транзакции в целом
            # и как время первой транзакции в новом периоде
            last_time, self.last_unix = derive_from_last_time(last_txn_unix=self.start_unix, lag_interval=lag_interval)
            self.start_unix = self.last_unix
            return last_time, self.last_unix

        # Если достигнут лимит исходящих транзакций для периода активности
        elif self.out_txns == self.out_lim:
            time_delta = self.get_time_delta(two_way=True)
            lag_interval = self.configs.lag_interval + time_delta
            # Входящие = 0, исходящие = 1
            self.txns_count(receive=receive, reset=True)
            # Создаем время и записываем его как время последней транзакции в целом
            # и как время первой транзакции в новом периоде
            last_time, self.last_unix = derive_from_last_time(last_txn_unix=self.start_unix, lag_interval=lag_interval)
            self.start_unix = self.last_unix
            return last_time, self.last_unix

        # Тоже дельта, но не может быть <= 0 т.к. тут мы ее используем как lag_interval
        # Это для случаев когда транзакция совершается в тот же период активности что и последняя
        time_delta = self.get_time_delta(two_way=False)
        print(time_delta)
        last_time, self.last_unix = derive_from_last_time(last_txn_unix=self.last_unix, lag_interval=time_delta)
        # +1 к счетчику соответствующего типа
        self.txns_count(receive=receive, reset=False)

        return last_time, self.last_unix
    

    def reset_cache(self):
        """
        Очистка кэшированных данных в атрибутах:
        self.start_unix
        self.last_unix
        self.in_txns
        self.out_txns
        """
        self.start_unix = 0
        self.last_unix = 0
        self.in_txns = 0
        self.out_txns = 0

**Тест `DropTimeHandeler`**

In [435]:
# Временный reload для разработки
import data_generator.utils
importlib.reload(data_generator.utils)
import data_generator.fraud.drops.time
importlib.reload(data_generator.fraud.drops.time)
from data_generator.utils import DropConfigs
from data_generator.fraud.drops.time import DropTimeHandler

drop_time_hanler1 = DropTimeHandler(configs=drop_configs)

In [389]:
two_way_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=True) for _ in range(1000)])
two_way_deltas1.mean(), two_way_deltas1.min(), two_way_deltas1.max()

(np.float64(1.25), np.int64(-180), np.int64(180))

In [390]:
pos_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=False) for _ in range(1000)])
pos_deltas1.mean(), pos_deltas1.min(), pos_deltas1.max()

(np.float64(105.04), np.int64(30), np.int64(180))

In [391]:
for _ in range(3):
    drop_time_hanler1.txns_count(receive=True, reset=False)
    drop_time_hanler1.txns_count(receive=False, reset=False)

In [392]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(3, 3)

In [393]:
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

In [394]:
drop_time_hanler1.in_txns = 2
drop_time_hanler1.out_txns = 4
drop_time_hanler1.txns_count(receive=False, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(0, 1)

**`DropTimeHandler.get_txn_time` тест**  
Тест-кейсы
- Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции
- Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- Не первая транзакция и достигнут лимит исходящих транзакций для периода активности
- Не первая транзакция. Лимиты не достигнуты. Транзакция входящая
- Не первая транзакция. Лимиты не достигнуты. Транзакция исходящая 

In [330]:
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
start_unix = pd_timestamp_to_unix(start_time)
start_time, start_unix

(Timestamp('2025-07-02 11:15:00'), 1751454900)

In [331]:
last_time = pd.to_datetime("2025-07-02 18:31:00", format="%Y-%m-%d %H:%M:%S")
last_unix = pd_timestamp_to_unix(last_time)
last_time, last_unix

(Timestamp('2025-07-02 18:31:00'), 1751481060)

1. Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции

In [407]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = 0
drop_time_hanler1.out_txns = 0
droptime1, dropunix1 = drop_time_hanler1.get_txn_time(receive=True, in_txns=0)
droptime1, dropunix1

(Timestamp('2025-01-18 05:35:00'), np.int64(1737178500))

In [408]:
drop_time_hanler1.last_unix, drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(np.int64(1737178500), 1, 0)

2. Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [379]:
start_time, last_time

(Timestamp('2025-07-02 11:15:00'), Timestamp('2025-07-02 18:31:00'))

In [426]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime2, dropunix2 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start2 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime2, dropunix2, droptime_start2

(Timestamp('2025-07-03 09:03:00'),
 1751533380,
 Timestamp('2025-07-03 09:03:00'))

In [427]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(0, 1, 1751533380, 1751533380)

In [428]:
droptime2 - start_time

Timedelta('0 days 21:48:00')

3. Не первая транзакция и достигнут лимит исходящих транзакций для периода активности  
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [446]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime3, dropunix3 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start3 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime3, dropunix3, droptime_start3

(Timestamp('2025-07-03 13:06:00'),
 1751547960,
 Timestamp('2025-07-03 13:06:00'))

In [447]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(0, 1, 1751547960, 1751547960)

In [448]:
droptime3 - start_time

Timedelta('1 days 01:51:00')

4. Не первая транзакция. Лимиты не достигнуты. Транзакция **входящая**

In [451]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime4, dropunix4 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start4 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime4, dropunix4, droptime_start4

In txns: 1 
Out txns: 4
126


(Timestamp('2025-07-02 20:37:00'),
 1751488620,
 Timestamp('2025-07-02 11:15:00'))

In [452]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(2, 4, 1751454900, 1751488620)

In [454]:
droptime4 - last_time

Timedelta('0 days 02:06:00')

5. Не первая транзакция. Лимиты не достигнуты. Транзакция **исходящая**

In [457]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime5, dropunix5 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start5 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime5, dropunix5, droptime_start5

In txns: 1 
Out txns: 4
172


(Timestamp('2025-07-02 21:23:00'),
 1751491380,
 Timestamp('2025-07-02 11:15:00'))

In [458]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 5, 1751454900, 1751491380)

In [459]:
droptime5 - last_time

Timedelta('0 days 02:52:00')

## Класс `CreateDropTxn`

In [270]:
class CreateDropTxn:
    """
    Создание транзакций дропа под разное поведение.
    """
    def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
                attempts=0):
        """
        timestamps - pd.DataFrame.
        trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
        drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
        in_txns - int. Количество входящих транзакций
        out_txns - int. Количество исходящих транзакций
        in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        last_txn - dict. Полные данные последней транзакции
        attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
        """
        self.timestamps = timestamps
        self.trans_partial_data = trans_partial_data
        self.drop_client = drop_client_cls
        self.in_txns = in_txns
        self.out_txns = out_txns
        self.in_lim = in_lim
        self.out_lim = out_lim
        self.last_txn = last_txn
        self.attempts = attempts



    
    def stop_after_decline(self, declined):
        """
        Будет ли дроп пытаться еще после отклоненной операции
        или остановится
        ---------------
        declined - bool. Отклонена ли операция. Подразумевается что последняя.
        """
        if not declined:
            return
        
        if self.attempts == 0:
            return True

        if self.attempts > 0:
            return False

            
    def attempts_after_decline(self, min=0, max=4):
        """
        Определение количества попыток после первой отклоненной транзакции
        ---------------
        min - int. Минимальное число попыток
        max - int. Максимальное число попыток.
        """
        self.attempts = np.random.randint(min, max + 1)
            
        
    def deduct_attempts(self, declined, receive):
        """
        Вычитание попытки операции совершенной при статусе declined
        ---------------
        declined - bool. Отклоняется ли текущая транзакция
        receive - bool. Является ли транзакция входящей
        """
        if self.attempts == 0:
            return 
            
        if declined and not receive:
            self.attempts -= 1
            
        
    def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False):
        """
        Один входящий/исходящий перевод либо одно снятие в банкомате.
        ---------------------
        online - bool. Онлайн перевод или снятие в банкомате.
        declined - bool. Будет ли текущая транзакция отклонена.
        in_chunks - bool. Транзакция будет частью серии транзакций.
        to_drop_share - float. Вероятность, что дроп пошлет другому дропу
        receive - входящий перевод или нет.
        """
        client_id = self.trans_partial_data.client_info.client_id # берем из namedtuple
        
        # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
        txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

        if receive:
            self.in_txns += 1
            amount = self.drop_client.receive(declined=declined)
            account = self.drop_client.account
            
        elif not receive and online:
            to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
            self.out_txns += 1
            account = self.drop_client.get_account(to_drop=to_drop)
            
        elif not receive and not online:
            account = self.drop_client.account
            self.out_txns += 1
        
            
        # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
        merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
                                                self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
        # Генерация суммы если исходящая транзакция
        # Если эта транзакция только часть серии операций для распределения всего баланса
        if in_chunks:
            chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
            amount = self.drop_client.one_operation(amount=chunk, declined=declined, in_chunks=in_chunks)
            
        # Иначе если не по частям и не входящая транзакция
        elif not in_chunks and not receive:
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

        if declined:
            status = "declined"
            is_fraud = True
            rule = "drop_flow_cashout"
        else:
            status = "approved"
            is_fraud = False
            rule = "not applicable"

        # Статичные характеристики
        is_suspicious = False
        category_name="not applicable"

        # Сборка всех данных в транзакцию и запись как послдней транзакции
        self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
                             category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
                             trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
                             is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

        return self.last_txn

    def limit_reached(self):
        """
        Проверка достижения лимитов входящих и исходящих транзакций
        Сверка с self.in_lim и self.out_lim
        ------------------------
        Вернет True если какой либо лимит достигнут
        """
        if self.in_lim == self.in_txns:
            return True
        if self.out_lim == self.out_txns:
            return True
        return False

    def reset_txn_counters(self, in_txns=False, out_txns=False, batch_txns=False):
        """
        Сброс счетчиков входящих и/или исходящих транзакций
        Партия денег это полученные деньги которые дроп должен распределить
        ------------------
        in_txns - bool. Сбросить счетчик входящих.
        out_txns - bool. Сбросить счетчик исходящих.
        batch_txns - bool. Сбросить счетчик транзакций партии денег.
        """
        if in_txns:
            self.in_txns = 0
        if out_txns:
            self.out_txns = 0
        if batch_txns:
            self.batch_txns = 0

    def reset_cache(self, only_counters=True):
        """
        Сброос кэшированных данных
        -------------
        only_counters - bool
                        Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
                        Если False то также сбросится информация о последней транзакции self.last_txn
        """
        
        self.in_txns = 0
        self.out_txns = 0
        self.attempts = 0

        if only_counters:
            return

        self.last_txn = {}

In [ ]:
# class CreateDropTxn:
#     """
#     Создание транзакций дропа под разное поведение.
#     """
#     def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
#                 attempts=0):
#         """
#         timestamps - pd.DataFrame.
#         trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
#         drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
#         in_txns - int. Количество входящих транзакций
#         out_txns - int. Количество исходящих транзакций
#         in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
#         out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
#         last_txn - dict. Полные данные последней транзакции
#         attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
#         """
#         self.timestamps = timestamps
#         self.trans_partial_data = trans_partial_data
#         self.drop_client = drop_client_cls
#         self.in_txns = in_txns
#         self.out_txns = out_txns
#         self.in_lim = in_lim
#         self.out_lim = out_lim
#         self.last_txn = last_txn
#         self.attempts = attempts

#     def get_time_delta(self, min=-180, max=180, minutes=True):
#         """
#         Получение случайного интервала времени в секундах или минутах из равномерного распределения
#         ---------------------
#         min - int. Минимальное возможное значение
#         max - int. Максимальное возможное значение
#         minutes - bool. Минуты или секунды
#         """
#         if minutes:
#             return round(np.random.uniform(min, max))
            
#         return round(np.random.uniform(min, max) * 60)

#     def get_txn_time(self, in_lim=2, out_lim=5, lag_interval=1440):
#         """
#         Генерация времени транзакции
#         ------------------
#         in_lim - int. Количество входящих транзакций после которых дроп уходит на паузу указанную в lag_interval
#                       Т.е. если на момент генерации времени уже сделано in_lim транзакций, то берется время последней
#                       транзакции и прибавляется указанный lag_interval +/- случайное число минут из delta.
#         out_lim - int. Количество исходящих транзакций после которых дроп уходит на паузу указанную в lag_interval
#         lag_interval - int. Желаемый лаг по времени от последней транзакции в минутах.
#                             Используется для перерывов в активности дропа. По умолчанию 1440 минут т.е. 24 часа
#         """
#         # Если это первая транзакция. Т.к. активность дропа начинается с входящей транзакции
#         if self.in_txns == 0:
#             time_sample = self.timestamps.sample(1)
#             txn_time = time_sample.timestamp.iat[0]
#             txn_unix = time_sample.unix_time.iat[0]
#             return txn_time, txn_unix

#         # Для последующих транзакций
#         last_txn_unix = self.last_txn["unix_time"]

#         # Если достигнуты лимиты активности дропа на период: входящих или исходящих транзакций
#         if self.in_txns == in_lim or self.out_txns == out_lim:
#             # Генерация дельты, чтобы время выглядело не ровным, а случайным.
#             # Слагаем её с lag_interval
#             time_delta = self.get_time_delta(min=-180, max=180)
#             lag_interval += time_delta
#             return derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=lag_interval)

#         # Тоже дельта, но не может быть <= 0 т.к. тут мы ее используем как lag_interval
#         # Это для случаев когда транзакция совершается в тот же период активности что и последняя
#         time_delta = self.get_time_delta(min=30, max=180)
        

#         return derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=time_delta)

    
#     def stop_after_decline(self, declined):
#         """
#         Будет ли дроп пытаться еще после отклоненной операции
#         или остановится
#         ---------------
#         declined - bool. Отклонена ли операция. Подразумевается что последняя.
#         """
#         if not declined:
#             return
        
#         if self.attempts == 0:
#             return True

#         if self.attempts > 0:
#             return False

            
#     def attempts_after_decline(self, min=0, max=4):
#         """
#         Определение количества попыток после первой отклоненной транзакции
#         ---------------
#         min - int. Минимальное число попыток
#         max - int. Максимальное число попыток.
#         """
#         self.attempts = np.random.randint(min, max + 1)
            
        
#     def deduct_attempts(self, declined, receive):
#         """
#         Вычитание попытки операции совершенной при статусе declined
#         ---------------
#         declined - bool. Отклоняется ли текущая транзакция
#         receive - bool. Является ли транзакция входящей
#         """
#         if self.attempts == 0:
#             return 
            
#         if declined and not receive:
#             self.attempts -= 1
            
        
#     def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False):
#         """
#         Один входящий/исходящий перевод либо одно снятие в банкомате.
#         ---------------------
#         online - bool. Онлайн перевод или снятие в банкомате.
#         declined - bool. Будет ли текущая транзакция отклонена.
#         in_chunks - bool. Транзакция будет частью серии транзакций.
#         to_drop_share - float. Вероятность, что дроп пошлет другому дропу
#         receive - входящий перевод или нет.
#         """
#         client_id = self.trans_partial_data.client_info.client_id
        
#         # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
#         txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

#         if receive:
#             self.in_txns += 1
#             amount = self.drop_client.receive(declined=declined)
#             account = self.drop_client.account
            
#         elif not receive and online:
#             to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
#             self.out_txns += 1
#             account = self.drop_client.get_account(to_drop=to_drop)
            
#         elif not receive and not online:
#             account = self.drop_client.account
#             self.out_txns += 1
        
            
#         # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
#         merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
#                                                 self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
#         # Генерация суммы если исходящая транзакция
#         # Если эта транзакция только часть серии операций для распределения всего баланса
#         if in_chunks:
#             chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
#             amount = self.drop_client.one_operation(amount=chunk, declined=declined, in_chunks=in_chunks)
            
#         # Иначе если не по частям и не входящая транзакция
#         elif not in_chunks and not receive:
#             amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

#         if declined:
#             status = "declined"
#             is_fraud = True
#             rule = "drop_flow_cashout"
#         else:
#             status = "approved"
#             is_fraud = False
#             rule = "not applicable"

#         # Статичные характеристики
#         is_suspicious = False
#         category_name="not applicable"

#         # Сборка всех данных в транзакцию и запись как послдней транзакции
#         self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
#                              category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
#                              trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
#                              is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

#         return self.last_txn

#     def limit_reached(self):
#         """
#         Проверка достижения лимитов входящих и исходящих транзакций
#         Сверка с self.in_lim и self.out_lim
#         ------------------------
#         Вернет True если какой либо лимит достигнут
#         """
#         if self.in_lim == self.in_txns:
#             return True
#         if self.out_lim == self.out_txns:
#             return True
#         return False

#     def reset_txn_counters(self, in_txns=False, out_txns=False, batch_txns=False):
#         """
#         Сброс счетчиков входящих и/или исходящих транзакций
#         Партия денег это полученные деньги которые дроп должен распределить
#         ------------------
#         in_txns - bool. Сбросить счетчик входящих.
#         out_txns - bool. Сбросить счетчик исходящих.
#         batch_txns - bool. Сбросить счетчик транзакций партии денег.
#         """
#         if in_txns:
#             self.in_txns = 0
#         if out_txns:
#             self.out_txns = 0
#         if batch_txns:
#             self.batch_txns = 0

#     def reset_cache(self, only_counters=True):
#         """
#         Сброос кэшированных данных
#         -------------
#         only_counters - bool
#                         Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
#                         Если False то также сбросится информация о последней транзакции self.last_txn
#         """
        
#         self.in_txns = 0
#         self.out_txns = 0
#         self.attempts = 0

#         if only_counters:
#             return

#         self.last_txn = {}

**`CreateDropTxn.single_operation` тест**

In [159]:
drop_stamps = create_timestamps_range_df(start="2025-01-01", end="2025-01-31")

In [260]:
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)

In [261]:
# drop_txn_part_data.client_info

In [262]:
drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

In [263]:
create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)

In [264]:
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, create_drop_txn_tst.last_txn, drop_client_test2.batch_txns

(0, 0, {}, 0)

In [1342]:
# ?CreateDropTxn.single_operation

Входящая НЕотклоненная транзакция дропу

In [205]:
create_drop_txn_tst.reset_cache(only_counters=False)
receive_txn = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
pd.DataFrame([receive_txn])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-02 12:17:00,1735820220,52000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable


In [206]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(52000.0), 1)

In [207]:
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, create_drop_txn_tst.attempts

(1, 0, 0)

In [1542]:
# create_drop_txn_tst.last_txn

НЕ отклоненный исходящий перевод целиком

In [208]:
whole_out = create_drop_txn_tst.single_operation(online=True, receive=False, declined=False, in_chunks=False)
pd.concat([pd.DataFrame([receive_txn]), pd.DataFrame([whole_out])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-02 12:17:00,1735820220,52000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-02 13:25:00,1735824300,52000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,21193,False,False,approved,not applicable


НЕ отклоненное снятие целиком

In [209]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn2 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
whole_atm = create_drop_txn_tst.single_operation(online=False, receive=False, declined=False, in_chunks=False)

pd.concat([pd.DataFrame([receive_txn2]), pd.DataFrame([whole_atm])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-24 01:00:00,1737680400,6500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-24 02:35:00,1737686100,6500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable


In [210]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(0.0), 1)

НЕ отклоненный исходящий перевод частями

In [267]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn3 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
pd.DataFrame([receive_txn3])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-22 12:27:00,1737548820,36500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable


In [268]:
all_txns3 = [receive_txn3]
while drop_client_test2.balance > 0:
    part_out = create_drop_txn_tst.single_operation(online=True, receive=False, declined=False, in_chunks=True)
    all_txns3.append(part_out)
pd.DataFrame(all_txns3)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-22 12:27:00,1737548820,36500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-22 15:20:00,1737559200,20000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,16461,False,False,approved,not applicable
2,1,2025-01-22 18:11:00,1737569460,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,18060,False,False,approved,not applicable
3,1,2025-01-22 20:14:00,1737576840,11500.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,19418,False,False,approved,not applicable


In [269]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(0.0), 3)

НЕ отклоненное снятие частями

In [1700]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn4 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)

all_txns4 = [receive_txn4]
while drop_client_test2.balance > 0:
    part_out = create_drop_txn_tst.single_operation(online=False, receive=False, declined=False, in_chunks=True)
    all_txns4.append(part_out)
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-17 08:20:00,1737102000,45000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-17 10:35:00,1737110100,22500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable
2,1,2025-01-17 11:40:00,1737114000,22500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable


Отклоненный входящий перевод

In [1706]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn5 = create_drop_txn_tst.single_operation(online=True, receive=True, declined=True, in_chunks=False)
pd.DataFrame([receive_txn5])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-01 07:56:00,1735718160,30000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,True,False,declined,client_is_drop


In [1707]:
drop_client_test2.balance, drop_client_test2.batch_txns

(0, 0)

Отклоненный исходящий перевод целиком

In [ ]:
online=True, receive=False, declined=True, in_chunks=False

In [1710]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn6 = create_drop_txn_tst.single_operation(online=True, receive=True, declined=False, in_chunks=False)

whole_out2 = create_drop_txn_tst.single_operation(online=True, receive=False, declined=True, in_chunks=False)


pd.concat([pd.DataFrame([receive_txn6]), pd.DataFrame([whole_out2])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-30 12:16:00,1738239360,25000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-30 14:51:00,1738248660,25000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,17922,True,False,declined,client_is_drop


In [1711]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(25000.0), 1)

**Отклоненное снятие целиком**

In [1781]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn7 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
whole_atm_2 = create_drop_txn_tst.single_operation(online=False, receive=False, declined=True, in_chunks=False)

pd.concat([pd.DataFrame([receive_txn7]), pd.DataFrame([whole_atm_2])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-03 14:40:00,1735915200,29500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-03 15:42:00,1735918920,29500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,True,False,declined,client_is_drop


In [1782]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(29500.0), 1)

**Отклоненный  исходящий перевод частями**

In [1794]:
# Объявление новых объектов классов для тестов.
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)


drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)
create_drop_txn_tst.reset_cache(only_counters=False)
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, drop_client_test2.batch_txns, create_drop_txn_tst.limit_reached()

(0, 0, 0, False)

In [1795]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
all_txns5 = []
declined=False
crypto_rate = 0.05
create_drop_txn_tst.attempts_after_decline(min=0, max=1)
drop_client_test2.chunk_size = 5000
drop_client_test2.batch_txns = 1

i = 1

while True:
    receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
    all_txns5.append(receive_txn8)
    if declined:
        break
        
    while drop_client_test2.balance > 0:
        if np.random.uniform(0, 1) < crypto_rate:
            part_out = create_drop_txn_tst.purchase(online=True, declined=declined, in_chunks=True, channel="crypto_exchange")
        else:
            part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
        all_txns5.append(part_out)
        print(f"iter {i}")
        create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
        if create_drop_txn_tst.stop_after_decline(declined=declined):
            break
           
        declined = create_drop_txn_tst.limit_reached()
        i += 1
        
    # pd.DataFrame(all_txns5)

iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9


In [1798]:
pd.DataFrame(all_txns5)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-26 19:17:00,1737919020,9000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-26 19:54:00,1737921240,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,17264,False,False,approved,not applicable
2,1,2025-01-26 22:17:00,1737929820,4000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,18967,False,False,approved,not applicable
3,1,2025-01-26 23:24:00,1737933840,32500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
4,1,2025-01-27 21:51:00,1738014660,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,20290,False,False,approved,not applicable
5,1,2025-01-28 19:29:00,1738092540,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,20575,False,False,approved,not applicable
6,1,2025-01-29 18:13:00,1738174380,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,19220,False,False,approved,not applicable
7,1,2025-01-30 16:10:00,1738253400,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,23528,False,False,approved,not applicable
8,1,2025-01-31 13:16:00,1738329360,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,24752,False,False,approved,not applicable
9,1,2025-02-01 10:57:00,1738407420,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,16527,False,False,approved,not applicable


np.float64(2500.0)

## Функция генерации транзакции дропа `gen_drop_txn`

In [1074]:
def one_batch_dist(drop_info, drop_client_act: DropClient, batch):
    """
    Распределение дропом одной партии полученных денег
    -------------------
    drop_info - namedtuples из itertuples(). Информация клиента-дропа из общего датафрейма клиентов
    drop_client_act - DropClient. Генератор атрибутов дропа и его действий; генератор счетов транзакций дропа
    batch - int. id партии денег полученной дропом.
    """
    amount_received = drop_client_act.received(batch=batch)
    scenario = np.random.choice(["atm-only", "transfer+atm", "transfer", "transfer_chunks"])
    for i in range(n):
        if i == 0:
            first = True
        else:
            first = False
        
        if scenario == "atm-only":
            atm_withdrawal()
            
        elif scenario == "atm+transfer":
            atm_withdrawal()

## Функция генерации активности дропа `drop_activity`

In [ ]:
def drop_activity(drop_info, drop_client_act: DropClient, lifetime):
    """
    drop_info - namedtuples из itertuples(). Информация клиента-дропа из общего датафрейма клиентов
    drop_client_act - DropClient. Генератор атрибутов дропа и его действий; генератор счетов транзакций дропа.
    lifetime - int. Сколько дней дроп будет проявлять активность, учитывая даже отклоненные транзакции
    """
    amount_received = drop_client_act.received(batch=)
    scenario = np.random.choice(["atm-only", "transfer+atm", "transfer", "transfer_chunks"])
    for i in range(n):
        if i == 0:
            first = True
        else:
            first = False
        
        if scenario == "atm-only":
            atm_withdrawal()
            
        elif scenario == "atm+transfer":
            atm_withdrawal()

In [1077]:
?DropClient.receive

Signature:
DropClient.receive(
    self,
    batch,
    low=5000,
    high=100000,
    mean=30000,
    std=20000,
    round=500,
)
Docstring:
Генерация суммы входящего перевода
--------------------------
batch - int. id текущей партии денег, которую он должен распределить (снять, перевести)
low - float. Минимальная сумма
high - float. Максимальная сумма
mean - float. Средняя сумма
std - float. Стандартное отклонение
round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце.
             При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1.
File:      c:\users\iaros\appdata\local\temp\ipykernel_7648\484414114.py
Type:      function

## Функция генерации входящих/исходящих переводов дропов

In [ ]:
def drop_transfer(type, first, configs: ConfigForTrans, fraud_amount: TransAmount, \
                 trans_part_data: FraudTransPartialData, last_txn_unix, accounts, lag_interval=None) -> dict:
    
	if type == 'inbound':
		fraud_amount.transfer(type=type)
		fraud_amount.balance += amount
		trans_part_data.original_data(online=True)
        
	elif type == 'outbound':
		TransAmount.transfer(type=type)
		TransAmount.balance -= amount
		trans_part_data.original_data(online=True)

	if first:
        timestamp_sample = configs.timestamps.sample(1)
		txn_time = timestamp_sample.timestamp.iat[0]
        txn_unix = timestamp_sample.unix_time.iat[0]
	else:
		txn_time, txn_unix = derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=lag_interval)

    account = accounts.sample(1).iat[0]

	build_trans()


In [820]:
?get_time_for_trans

Signature:
get_time_for_trans(
    trans_df,
    is_fraud,
    time_weights,
    timestamps,
    timestamps_1st_month,
    round_clock,
    online=None,
    rule=None,
    geo_distance=None,
    lag=None,
)
Docstring:
trans_df - датафрейм с транзакциями текущего клиента. Откуда брать информацию по предыдущим транзакциям клиента
is_fraud - boolean. Фрод или не фрод
time_weights - датафрейм с весами часов в периоде времени
timestamps - датафрейм с timestamps
timestamps_1st_month - сабсет timestamps отфильтрованный по первому месяцу и, если применимо, году. Чтобы генерировать первые транзакции
round_clock - boolean. Круглосуточная или дневная категория.
online - boolean. Онлайн или оффлайн покупка. True or False
rule - str. Название антифрод правила
geo_distance - int. Дистанция между локацией последней и текущей транзакции если фрод со сменой геолокации - в километрах
lag - boolean. Задержка по времени от предыдущей транзакции. Нужна для моделирования увеличения частоты транзакций.
     

# Урезание количества клиентов - `отменено`
**Думаю просто можно равновероятно семплировать n клиентов и все, без ухищрений как внизу**  
Т.к. на ~5400 клиентов нужно очень много транзакций, чтобы у каждого их было хотя бы 50.  
Условно 270000 транзакций по 50 на 5400 клиентов. Скорее всего генерироваться будут больше часа

In [436]:
# на данный момент примерно до 800.

# clients_reduced_num = pd.DataFrame(districts_ru.clients.mul(0.15).round().astype("int"))
# clients_reduced_num["city"] = districts_ru["area"].copy()
# clients_reduced_num

# clients_reduced_df = gpd.GeoDataFrame(columns=clients_with_geo.columns)
# # clients_reduced_df.columns = 
# for col, data_type in clients_with_geo.dtypes.items():
#     # if data_type != "geometry" print(data_type)
#     clients_reduced_df[col] = clients_reduced_df[col].astype(f"{data_type}")
# # list(zip(clients_with_geo.dtypes.items(), clients_reduced_df.columns))
# clients_reduced_df.dtypes

# for row in clients_reduced_num.itertuples():
#     clients_with_geo_part = clients_with_geo.loc[clients_with_geo["area"] == row.city].iloc[0:row.clients]
#     clients_reduced_df = pd.concat([clients_reduced_df,clients_with_geo_part])

# clients_reduced_df

# Старый код, функции и т.д.

# Функция генерации весов для часов в периоде времени `get_time_for_trans` - Не модульная. Старая версия

In [ ]:
# def get_time_for_trans(trans_df, client_id, is_fraud, time_weights, timestamps, online=None, rule=None, geo_distance=None, lag=None):
#     """
#     trans_df - датафрейм с транзакциями. Откуда брать информацию по предыдущим транзакциям
#     client_id - id клиента, число, клиент чьи транзакции проверяются
#     is_fraud - boolean. Фрод или не фрод
#     time_weights - датафрейм с весами часов в периоде времени
#     timestamps - датафрейм с timestamps
#     online - boolean. Онлайн или оффлайн покупка. True or False
#     rule - строка. Название антифрод правила
#     geo_distance - число. Дистанция между локацией последней и текущей транзакции если фрод со сменой геолокации - в метрах
#     lag - boolean. Задержка по времени от предыдущей транзакции. Нужна для моделирования увеличения частоты транзакций.
#           Это задержка именно между последней легитимной транзакцией и серией частых транзакций. Подразумевается что функция
#           get_time_for_trans будет использована в цикле, и для первой итерации lag будет True.

#     Возвращает время для генерируемой транзакции в виде pd.Timestamp и в виде unix времени
#     """
    
#     # timestamp последней транзакции клиента - может быть вынести фильтрацию по client_id из функции?
#     # Хотя наверное и вне функции все равно надо будет каждый раз обновлять состояние перед генерацией новой транзакции
    
#     last_txn_time = trans_df[trans_df.client_id == client_id].txn_time.max()
#     last_txn_unix = trans_df[trans_df.client_id == client_id].unix_time.max()
    
#     # Если нет предыдущей транзакции т.е. нет последнего времени
#     if not is_fraud and last_txn_time is pd.NaT:
#         # берем первый год всего времененного периода т.к. это первая транзакция, пусть будет создана в первом году
#         timestamps_1st_year = timestamps.loc[timestamps.timestamp.dt.year == timestamps.timestamp.dt.year.min()]

#         # семплируем час из весов времени, указав веса для семплирования
#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         # фильтруем основной датафрейм с диапазоном таймстемпов по этому часу
#         timestamps_subset = timestamps_1st_year.loc[timestamps_1st_year.hour == txn_hour]
#         # из отфильтрованного датафрейма таймстемпов семплируем один таймстемп с равной вероятностью
#         txn_time = timestamps_subset.timestamp.sample(n=1, replace=True).iloc[0]
#         txn_unix = (txn_time - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

#     # Если есть предыдущая транзакция

#     # Не фрод. Но есть предыдущая транзакция
#     elif not is_fraud:

#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
#         timestamp_sample = timestamps_subset.sample(n=1, replace=True)
#         trans_time_diff = timestamp_sample.unix_time - last_txn_unix
#         pos_one_hour_diff = 3600
#         neg_one_hour_diff = -3600
        
#         # если время между текущей и последней транзакцией меньше часа в положительную сторону,
#         # то увеличим время чтобы разница была минимум час 
#         if trans_time_diff < pos_one_hour_diff and trans_time_diff >= 0:
#             time_addition = pos_one_hour_diff - trans_time_diff
#             txn_unix = txn_unix + time_addition
#             txn_time = pd.to_datetime(txn_unix, unit="s")
        
#         # если время между текущей и последней транзакцией меньше часа в отрицательную сторону, 
#         # то уменьшим время чтобы разница была минимум час
#         elif trans_time_diff > neg_one_hour_diff and trans_time_diff < 0:
#             time_subtraction = neg_one_hour_diff - trans_time_diff
#             txn_unix = txn_unix + time_subtraction
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Фрод. Правила: другая гео за короткое время либо по локации оффлайн мерчанта либо по новому ip адресу
#     elif is_fraud and rule in ["fast_geo_change", "fast_geo_change_online"]:
#         # Выставим порог скорости перемещения между точками транзакций - м/с
#         # выше порога - детект как фрода. 
#         # Это нужно чтобы генерировать соответствующее время в зависимости от дистанции между точками транзакций
#         # пусть будет порог в 800 км/ч. Делим на 3.6 для перевода в м/с
#         speed_threshold = 800 / 3.6
        
#         # Случайно сгенерированная фактическая скорость превышающая легитимный порог. Допустим от 801 до 36000 км/ч
#         # до 36000 км/ч т.к. грубо говоря транзакция может быть совершена через 20 минут в 9000 км от предыдущей т.е. как-будто бы скорость 36000 км/ч
#         # 9000 км взяты как весьма примерное, самое длинное возможное расстояние между городами России при путешествии самолетом.
#         # но в зависимости от расстояния мы берем разные границы для распределений, чтобы не было перекоса в очень быстрое время. 
#         # Также 20 минут я случайно взял как средний интервал для подобной фрод транзакции.
#         # Конечно же "скорость перемещения" может быть и больше в реальной жизни
        
#         if geo_distance < 1000_000:
#             fact_speed = random.randint(speed_threshold + 1, 3000) / 3.6
#         elif geo_distance >= 1000_000 and geo_distance <= 3000_000:
#             fact_speed = random.randint(speed_threshold + 1, 9000) / 3.6
#         elif geo_distance > 3000_000 and geo_distance <= 6000_000:
#             fact_speed = random.randint(speed_threshold + 1, 18000) / 3.6
#         else:
#             fact_speed = random.randint(speed_threshold + 1, 36000) / 3.6

#         # интервал времени между последней транзакцией и текущей фрод транзакцией
#         time_interval = geo_distance / fact_speed
        
#         txn_unix = last_txn_unix + time_interval
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Фрод. Увеличение количества транзакций в единицу времени выше установленного порога в процентах.
#     # генерируем время
#     elif is_fraud and rule == "trans_freq_increase":
#         # семплируем таймстемп из таймстемпов по времени не ранее последней транзакции
#         timestamp_sample = timestamps.loc[timestamps.timestamp > last_txn_time].sample(n=1, replace=True)
#         trans_time_diff = timestamp_sample.unix_time - last_txn_unix
#         lag_interval = 1800

#         # частота фрод транзакций. от 1 до 5 минут. Выразим в секундах для удобства
#         freq = random.randint(1, 5) * 60
        
#         # если транзакция первая в серии фрод транзакций - аргумент lag=True
#         # и интервал между последней транзакцией менее 30 минут
#         # прибавить интервал 30 минут к семплированному времени текущей транзакции
#         if lag and trans_time_diff < lag_interval:
#             txn_unix = timestamp_sample.unix_time + lag_interval
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#         # Если lag=True, но не надо добавлять интервал
#         elif lag:
#             txn_unix = timestamp_sample.unix_time
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#         # для остальных случаев - когда это не первые фрод транзакции в серии
#         else:
#             txn_unix = last_txn_unix + freq
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Любой другой фрод
#     elif is_fraud:

#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
#         timestamp_sample = timestamps_subset.sample(n=1, replace=True)

#         txn_unix = timestamp_sample.unix_time
#         txn_time = timestamp_sample.time

    
#     # время транзакции в виде timestamp и unix time
#     return txn_time, txn_unix
    

## Подфункция `ensure_min_interval_from_last_offline_trans` - когда есть предыдущие оффлайн транзакции и нужно обеспечить минимальный фиксированный интервал между транзакциями<br>
`Отменена`

In [ ]:
# def ensure_min_interval_from_last_offline_trans(last_txn_unix, timestamp_sample, time_diff=3600):
#     """
#     Обеспечивает минимальный фиксированный интервал от последней транзакции
#     -----------------------------------------------
#     last_txn_unix - unix время последней транзакции
#     timestamp_sample - случайно выбранная запись из датафрейма с таймстемпами
#     time_diff - желаемая минимальная разница от последней транзакции
    
#     Если сгенерированное время отличается от времени последней по времени транзакции на разницу меньше установленной
#     То вернет время с минимальной установленной разницей от последней транзакции.
#     Например либо час вперед либо час назад.
    
#     Если сгенерированное время раньше последней транзакции менее чем на time_diff, то вернет время последней транзакции минус time_diff.
#     Т.е. например на час раньше последней.
#     Если сгенерированное время позже последней транзакции менее чем на time_diff, то вернет время позже последней транзакции плюс time_diff.
#     Т.е. например на час позже последней.
#     Если разница в положительную либо в отрицательную сторону больше time_diff, то вернет исходное время без изменений.
#     """
    
#     timestamp_unix = timestamp_sample.unix_time.iloc[0]
#     trans_time_diff = timestamp_unix - last_txn_unix
#     pos_one_hour_diff = time_diff
#     neg_one_hour_diff = - time_diff
    
#     # если время между текущей и последней транзакцией меньше time_diff в положительную сторону,
#     # то увеличим время чтобы разница была минимум time_diff 
#     if trans_time_diff < pos_one_hour_diff and trans_time_diff >= 0:
#         print("Condition #1")
#         time_addition = pos_one_hour_diff - trans_time_diff
#         txn_unix = timestamp_unix + time_addition
#         txn_time = pd.to_datetime(txn_unix, unit="s")
    
#     # если время между текущей и последней транзакцией меньше time_diff в отрицательную сторону, 
#     # то уменьшим время чтобы разница была минимум time_diff
#     elif trans_time_diff > neg_one_hour_diff and trans_time_diff < 0:
#         print("Condition #2")
#         time_subtraction = neg_one_hour_diff - trans_time_diff
#         txn_unix = timestamp_unix + time_subtraction
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # если разница во времени больше установленной
#     else:
#         txn_unix = timestamp_unix
#         txn_time = pd.to_datetime(txn_unix, unit="s")
        
        
#     return txn_time, txn_unix

In [ ]:
# def check_min_interval_from_near_trans(client_txns, timestamp_sample, online, offline_time_diff=60, \
#                                        online_time_diff=6, online_ceil=60, general_diff=30, general_ceil=90):
#     """
#     Если для сгенерированного времени есть транзакции, которые по времени ближе заданного минимума, 
#     то создать время на основании времени последней транзакции + установленный минимальный интервал.
#     Проверяет время либо для оффлайн либо для онлайн транзакций.
#     Для оффлайн и онлайн транзакций можно поставить свои минимальные интервалы.
#     Для онлайн ставится и максимальный интервал. Время берется случайно и равновероятно из диапазона между online_time_diff и online_ceil
#     -----------------------------------------------

#     client_txns - датафрейм с транзакциями клиента.
#     timestamp_sample - случайно выбранная запись из датафрейма с таймстемпами
#     offline_time_diff - int. желаемая минимальная разница от последней оффлайн транзакции в минутах
#     online_time_diff - int. желаемая минимальная разница от последней оффлайн транзакции в минутах
#     online_ceil - int. желаемая максимальная разница от последней оффлайн транзакции в минутах
#     general_diff - int. минимально допустимая разница от последней транзакции противоположной по флагу online.
#     general_ceil - int. максимальная разница от последней транзакции противоположной по флагу online.
#                    При случае необходимости увеличения разрыва времени.
#     """
    
#     timestamp_unix = timestamp_sample.unix_time.iloc[0]

#     offline_txns = client_txns.loc[client_txns.online == False]
#     online_txns = client_txns.loc[client_txns.online == True]
    
#     # смотрим ближайшие оффлайн и онлайн транзакции по времени по модулю.
#     # В переменные записываем время по модулю до ближайшей транзакции онлайн или оффлайн типа
#     closest_offline_txn = offline_txns.unix_time.sub(timestamp_unix).abs().min()
#     closest_online_txn = online_txns.unix_time.sub(timestamp_unix).abs().min()

#     # перевод аргументов в секунды для работы с unix time
#     offline_time_diff= offline_time_diff * 60
#     online_time_diff= online_time_diff * 60
#     online_ceil= online_ceil * 60
#     general_diff = general_diff * 60
#     general_ceil = general_ceil * 60
    
#     # Если текущая транзакция - оффлайн и разница с ближайшей транзакцией меньше допустимой
#     if not online and closest_txn_by_time < offline_time_diff:
        
#         # извлекаем время последней оффлайн транзакции и прибавляем установленный интервал
#         last_offline_txn_unix = offline_txns.unix_time.max()
#         txn_unix = last_offline_txn_unix + offline_time_diff
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Если текущая транзакция - онлайн и разница с ближайшей транзакцией меньше допустимой
#     elif online and closest_txn_by_time < online_time_diff:
        
#         # извлекаем время последней онлайн транзакции и прибавляем установленный интервал
#         last_online_txn_unix = online_txns.unix_time.max()

#         # Берем случайное время между online_time_diff и online_ceil
#         online_random_diff = random.randint(online_time_diff, online_ceil)

#         # прибавляем установленный интервал ко времени последней транзакции
#         txn_unix = last_online_txn_unix + online_random_diff
#         txn_time = pd.to_datetime(txn_unix, unit="s")
        
#     # если разница во времени является допустимой. Просто берем текущий timestamp
#     else:
#         txn_unix = timestamp_unix
#         txn_time = pd.to_datetime(txn_unix, unit="s")  
        
    
#     # Проверка есть ли слишком близкие по времени транзакции противоположные по флагу online
#     # Любая последняя транзакция. Если последняя транзакция такая же по флагу online, то разница
#     # Уже проверена и обеспечена в прошлом if-else блоке, и ничего не изменится.
#     last_txn_unix = client_txns.unix_time.max()
#     last_txn_diff = abs(last_txn_unix - txn_unix)
    
#     if last_txn_diff < general_diff:
#         general_random_diff = random.randint(general_diff, general_ceil)
#         txn_unix = last_txn_unix + general_random_diff
#         txn_time = pd.to_datetime(txn_unix, unit="s")
        
#     return txn_time, txn_unix

# Немодульная `get_time_for_trans` - старая версия

# `check_min_interval_from_near_trans` - до рефактора и добавления условий

In [ ]:
# def get_time_for_trans(trans_df, client_id, is_fraud, time_weights, timestamps, online=None, rule=None, geo_distance=None, lag=None):
#     """
#     trans_df - датафрейм с транзакциями. Откуда брать информацию по предыдущим транзакциям
#     client_id - id клиента, число, клиент чьи транзакции проверяются
#     is_fraud - boolean. Фрод или не фрод
#     time_weights - датафрейм с весами часов в периоде времени
#     timestamps - датафрейм с timestamps
#     online - boolean. Онлайн или оффлайн покупка. True or False
#     rule - строка. Название антифрод правила
#     geo_distance - число. Дистанция между локацией последней и текущей транзакции если фрод со сменой геолокации - в метрах
#     lag - boolean. Задержка по времени от предыдущей транзакции. Нужна для моделирования увеличения частоты транзакций.
#           Это задержка именно между последней легитимной транзакцией и серией частых транзакций. Подразумевается что функция
#           get_time_for_trans будет использована в цикле, и для первой итерации lag будет True.

#     Возвращает время для генерируемой транзакции в виде pd.Timestamp и в виде unix времени
#     """
    
#     # timestamp последней транзакции клиента - может быть вынести фильтрацию по client_id из функции?
#     # Хотя наверное и вне функции все равно надо будет каждый раз обновлять состояние перед генерацией новой транзакции
    
#     last_txn_time = trans_df[trans_df.client_id == client_id].txn_time.max()
#     last_txn_unix = trans_df[trans_df.client_id == client_id].unix_time.max()
    
#     # Если нет предыдущей транзакции т.е. нет последнего времени
#     if not is_fraud and last_txn_time is pd.NaT:
#         # берем первый год всего времененного периода т.к. это первая транзакция, пусть будет создана в первом году
#         timestamps_1st_year = timestamps.loc[timestamps.timestamp.dt.year == timestamps.timestamp.dt.year.min()]

#         # семплируем час из весов времени, указав веса для семплирования
#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         # фильтруем основной датафрейм с диапазоном таймстемпов по этому часу
#         timestamps_subset = timestamps_1st_year.loc[timestamps_1st_year.hour == txn_hour]
#         # из отфильтрованного датафрейма таймстемпов семплируем один таймстемп с равной вероятностью
#         txn_time = timestamps_subset.timestamp.sample(n=1, replace=True).iloc[0]
#         txn_unix = (txn_time - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

#     # Если есть предыдущая транзакция

#     # Не фрод. Но есть предыдущая транзакция
#     elif not is_fraud:

#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
#         timestamp_sample = timestamps_subset.sample(n=1, replace=True)
#         trans_time_diff = timestamp_sample.unix_time - last_txn_unix
#         pos_one_hour_diff = 3600
#         neg_one_hour_diff = -3600
        
#         # если время между текущей и последней транзакцией меньше часа в положительную сторону,
#         # то увеличим время чтобы разница была минимум час 
#         if trans_time_diff < pos_one_hour_diff and trans_time_diff >= 0:
#             time_addition = pos_one_hour_diff - trans_time_diff
#             txn_unix = txn_unix + time_addition
#             txn_time = pd.to_datetime(txn_unix, unit="s")
        
#         # если время между текущей и последней транзакцией меньше часа в отрицательную сторону, 
#         # то уменьшим время чтобы разница была минимум час
#         elif trans_time_diff > neg_one_hour_diff and trans_time_diff < 0:
#             time_subtraction = neg_one_hour_diff - trans_time_diff
#             txn_unix = txn_unix + time_subtraction
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Фрод. Правила: другая гео за короткое время либо по локации оффлайн мерчанта либо по новому ip адресу
#     elif is_fraud and rule in ["fast_geo_change", "fast_geo_change_online"]:
#         # Выставим порог скорости перемещения между точками транзакций - м/с
#         # выше порога - детект как фрода. 
#         # Это нужно чтобы генерировать соответствующее время в зависимости от дистанции между точками транзакций
#         # пусть будет порог в 800 км/ч. Делим на 3.6 для перевода в м/с
#         speed_threshold = 800 / 3.6
        
#         # Случайно сгенерированная фактическая скорость превышающая легитимный порог. Допустим от 801 до 36000 км/ч
#         # до 36000 км/ч т.к. грубо говоря транзакция может быть совершена через 20 минут в 9000 км от предыдущей т.е. как-будто бы скорость 36000 км/ч
#         # 9000 км взяты как весьма примерное, самое длинное возможное расстояние между городами России при путешествии самолетом.
#         # но в зависимости от расстояния мы берем разные границы для распределений, чтобы не было перекоса в очень быстрое время. 
#         # Также 20 минут я случайно взял как средний интервал для подобной фрод транзакции.
#         # Конечно же "скорость перемещения" может быть и больше в реальной жизни
        
#         if geo_distance < 1000_000:
#             fact_speed = random.randint(speed_threshold + 1, 3000) / 3.6
#         elif geo_distance >= 1000_000 and geo_distance <= 3000_000:
#             fact_speed = random.randint(speed_threshold + 1, 9000) / 3.6
#         elif geo_distance > 3000_000 and geo_distance <= 6000_000:
#             fact_speed = random.randint(speed_threshold + 1, 18000) / 3.6
#         else:
#             fact_speed = random.randint(speed_threshold + 1, 36000) / 3.6

#         # интервал времени между последней транзакцией и текущей фрод транзакцией
#         time_interval = geo_distance / fact_speed
        
#         txn_unix = last_txn_unix + time_interval
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Фрод. Увеличение количества транзакций в единицу времени выше установленного порога в процентах.
#     # генерируем время
#     elif is_fraud and rule == "trans_freq_increase":
#         # семплируем таймстемп из таймстемпов по времени не ранее последней транзакции
#         timestamp_sample = timestamps.loc[timestamps.timestamp > last_txn_time].sample(n=1, replace=True)
#         trans_time_diff = timestamp_sample.unix_time - last_txn_unix
#         lag_interval = 1800

#         # частота фрод транзакций. от 1 до 5 минут. Выразим в секундах для удобства
#         freq = random.randint(1, 5) * 60
        
#         # если транзакция первая в серии фрод транзакций - аргумент lag=True
#         # и интервал между последней транзакцией менее 30 минут
#         # прибавить интервал 30 минут к семплированному времени текущей транзакции
#         if lag and trans_time_diff < lag_interval:
#             txn_unix = timestamp_sample.unix_time + lag_interval
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#         # Если lag=True, но не надо добавлять интервал
#         elif lag:
#             txn_unix = timestamp_sample.unix_time
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#         # для остальных случаев - когда это не первые фрод транзакции в серии
#         else:
#             txn_unix = last_txn_unix + freq
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Любой другой фрод
#     elif is_fraud:

#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
#         timestamp_sample = timestamps_subset.sample(n=1, replace=True)

#         txn_unix = timestamp_sample.unix_time
#         txn_time = timestamp_sample.time

    
#     # время транзакции в виде timestamp и unix time
#     return txn_time, txn_unix
    

# Функция генератор одной легитимной POS транзакции `generate_one_legit_trans` - `первые тест сделаны` версия до рефактора

In [ ]:
# def generate_one_legit_trans(client_info, client_trans_df, category, merchants_df, online_merchant_ids, timestamps, \
#                              timestamps_1st_month, time_weights_dict):
#     """
#     Генерация одной легальной транзакции для клиента
#     ------------------------------------------------
#     client_info - namedtuple, полученная в результате итерации с помощью .itertuples() через датафрейм с информацией о клиентах
#     client_trans_df - датафрейм с транзакциями клиента.
#     category - датафрейм в одну запись с категорией и её характеристиками
#     merchants_df - датафрейм с оффлайн мерчантами заранее отфильтрованный по городу клиента т.к. это легальные транзакции
#     online_merchant_ids - id для онлайн мерчантов
#     time_weights_dict - веса для часов времени в виде словаря с: датафреймом с весами, названием распределения и цветом для графика
#     """

#     # Записываем данные клиента в переменные
#     client_id = client_info.client_id
#     client_area = client_info.area
#     client_area_lat = client_info.lat
#     client_area_lon = client_info.lon
#     client_ip = client_info.home_ip
    
    
#     # category = categories.sample(1, replace=True, weights=categories.share)
#     category_name = category["category"].iloc[0]
#     round_clock = category["round_clock"].iloc[0]
#     online = category["online"].iloc[0]
#     # средняя сумма для этой категории
#     amt_mean = category["avg_amt"].iloc[0]
#     # стандартное отклонение сумм для этой категории
#     amt_std = category["amt_std"].iloc[0]
    
#     # случайно сгенерированная сумма транзакции, но не менее 1
#     amount = max(1, np.random.normal(amt_mean, amt_std))

    
#     # 1. Offline_24h_Legit - круглосуточные оффлайн покупки
#     if not online and round_clock:
#         time_weights = time_weights_dict["Offline_24h_Legit"]["weights"]
#         merchant = merchants_df.loc[merchants_df.category == category_name].sample(1, replace=True)
#         merchant_id = merchant["merchant_id"].iloc[0]
#         trans_lat = merchant["merchant_lat"].iloc[0]
#         trans_lon = merchant["merchant_lon"].iloc[0]
#         status = "approved"
        
#         txn_time, txn_unix = get_time_for_trans(trans_df=client_trans_df, is_fraud=False, time_weights=time_weights, \
#                                                 timestamps=timestamps, timestamps_1st_month=timestamps_1st_month, \
#                                                 online=online)
#         pos_txn = {
#                     "client_id": client_id, "txn_time": txn_time, "unix_time":txn_unix, "amount": round(amount, 2), "type": "purchase",
#                     "channel": "POS", "category": category_name, "online": online, "merchant_id": merchant_id,
#                     "trans_city":client_area, "trans_lat": trans_lat, "trans_lon": trans_lon,"trans_ip":"not applicable",
#                     "account_to": np.nan, "is_fraud": False, "is_suspicious":False, "status":"approved",
#                     "rule":"not applicable"
#                     }


#     # 2. Online_Legit - Онлайн покупки
#     elif online:
#         time_weights = time_weights_dict["Online_Legit"]["weights"]
#         merchant_id = online_merchant_ids.sample(n=1).iloc[0]
#         # локация клиента по IP. Т.к. это не фрод. Просто записываем координаты города клиента
#         trans_lat = client_area_lat
#         trans_lon = client_area_lon
    
#         txn_time, txn_unix = get_time_for_trans(trans_df=client_trans_df, is_fraud=False, time_weights=time_weights, \
#                                                 timestamps=timestamps, timestamps_1st_month=timestamps_1st_month, \
#                                                 online=online)
#         pos_txn = {
#                     "client_id": client_id, "txn_time": txn_time, "unix_time":txn_unix, "amount": round(amount, 2), "type": "purchase",
#                     "channel": "POS", "category": category_name, "online": online, "merchant_id": merchant_id,
#                     "trans_city":client_area, "trans_lat": trans_lat, "trans_lon": trans_lon, "trans_ip":client_ip,
#                     "account_to": np.nan, "is_fraud": False, "is_suspicious":False, "status":"approved",
#                     "rule":"not applicable"
#                     }

#     # 3. Offline_Day_Legit - Оффлайн покупки. Дневные категории.
#     elif not online and not round_clock:
#         time_weights = time_weights_dict["Offline_Day_Legit"]["weights"]
#         merchant = merchants_df.loc[merchants_df.category == category_name].sample(1, replace=True)
#         merchant_id = merchant["merchant_id"].iloc[0]
#         trans_lat = merchant["merchant_lat"].iloc[0]
#         trans_lon = merchant["merchant_lon"].iloc[0]
#         status = "approved"
        
#         txn_time, txn_unix = get_time_for_trans(trans_df=client_trans_df, is_fraud=False, time_weights=time_weights, \
#                                                 timestamps=timestamps, timestamps_1st_month=timestamps_1st_month, \
#                                                 online=online)
#         pos_txn = {
#                     "client_id": client_id, "txn_time": txn_time, "unix_time":txn_unix, "amount": round(amount, 2), "type": "purchase",
#                     "channel": "POS", "category": category_name, "online": online, "merchant_id": merchant_id,
#                     "trans_city":client_area, "trans_lat": trans_lat, "trans_lon": trans_lon,"trans_ip":"not applicable",
#                     "account_to": np.nan, "is_fraud": False, "is_suspicious":False, "status":"approved",
#                     "rule":"not applicable"
#                     }


#     return pos_txn

# Основная функция генератор POS транзакций - изначальная. - старая, самая первая версия. До разбивки на легальные и фрод

In [ ]:
# Генерация POS транзакций
# def generate_legit_pos_transactions(client_info, start_date, end_date, time_weights_dict, categories, rules, trans_df, merchants_df, \
#                               area_centers, fraud_ips, online_merchant_ids, num=50, fraud_rate=0.01):
#     """
#     client_info  - датафрейм с данными клиента
#     start_date - первая транзакция
#     end_date - последняя транзакция
#     time_weights_dict - веса для часов времени в виде словаря с: датафреймом с весами, названием распределения и цветом для графика
#     categories - датафрейм с категориями и их характеристиками
#     rules -  датафрейм с антифрод правилами и их весами fraud_ips - датафрейм с фродовыми ip адресами
#     trans_df - датафрейм с транзакциями.
#                Куда их писать и куда обращаться за информацией по предыдущим транзакциям
#     merchants_df - датафрейм с оффлайн мерачантами
#     area_centers - названия городов с координатами центра
#     online_merchant_ids - id для онлайн мерчантов
#     num - количество транзакций на клиента
#     fraud_rate - доля фрода во всех транзакциях
#     """

#     # Датафрейм timestamp-ов откуда будет семплироваться время
#     timestamps = pd.DataFrame(pd.Series(pd.date_range(pd.to_datetime(start_date, format="%Y-%m-%d"), \
#                                                   pd.to_datetime(end_date, format="%Y-%m-%d"), freq='min'), name="timestamp"))
#     timestamps["hour"] = timestamps.timestamp.dt.hour
#     timestamps["unix_time"] = (timestamps.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#     # timestamps = pd.Series(pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date), freq='min'))
    
    



#     # итерируемся через уникальных клиентов
#     for row in client_info.itertuples():
#         pos_txns = [] # сюда временно будут добавляться созданные транзакции для текущего клиента

#         # данные клиента
#         client_id = row.client_id
#         client_area = row.area
#         client_area_geometry = row.geometry
#         client_ip = row.home_ip
        
#         # цикл создания n транзакций для текущего клиента
#         for _ in range(num):

#             # будет ли транзакция фродом или нет. Процент фрод транзакций зависит от выставленного fraud rate
#             is_fraud = random.random() < fraud_rate
            
#             #########################
#             # ЛЕГЕТИМНЫЕ ТРАНЗАКЦИИ #
#             #########################
#             if not is_fraud:
#                 # случайный выбор категории транзакции. В зависимости от частоты категории
#                 category = categories.sample(1, replace=True, weights=categories.share)
#                 category_name = category["category"].iloc[0]
#                 round_clock = category["round_clock"].iloc[0]
#                 online = category["online"].iloc[0]
#                 # средняя сумма для этой категории
#                 amt_mean = category["avg_amt"].iloc[0]
#                 # стандартное отклонение сумм для этой категории
#                 amt_std = category["amt_std"].iloc[0]
#                 # случайно сгенерированная сумма транзакции, но не менее 1
#                 amount = max(1, np.random.normal(amt_mean, amt_std))

#             # СТРОКА ДЛЯ ТЕСТОВ
#             # print(f"is_fraud:{is_fraud}, category_name: {category_name}")

            
#             # 1. Offline_24h_Legit - круглосуточные оффлайн покупки - НЕ фрод
#             if not is_fraud and not online and round_clock:
#                 time_weights = time_weights_dict["Offline_24h_Legit"]["weights"]
#                 merchant = merchants_df.loc[(merchants_df["area"] == client_area) & (merchants_df.category == category_name)].sample(1, replace=True)
#                 merchant_id = merchant["merchant_id"].iloc[0]
#                 trans_loc = merchant["geometry"].iloc[0]
#                 status = "approved"
                
#                 txn_time, txn_unix = get_time_for_trans(trans_df=trans_df, client_id=client_id, is_fraud=is_fraud, time_weights=time_weights, \
#                                                         timestamps=timestamps, online=False)
#                 pos_txns.append({
#                 "client_id": client_id, "txn_time": txn_time, "unix_time":txn_unix, "amount": round(amount, 2), "type": "purchase",
#                 "channel": "POS", "category": category_name, "online": online, "merchant_id": merchant_id,
#                 "trans_loc": trans_loc, "client_ip":"not applicable",
#                 "account_to": np.nan, "is_fraud": is_fraud, "status":"approved"
#             })


#             # 2. Online_Legit - Онлайн покупки - НЕ фрод
#             elif not is_fraud and online:
#                 time_weights = time_weights_dict["Online_Legit"]["weights"]
#                 merchant_id = online_merchant_ids.sample(n=1).iloc[0]
#                 # локация клиента по IP. Т.к. это не фрод. То просто ищем координаты города по городу клиента
#                 trans_loc = area_centers.loc[area_centers["area"] == client_area]

#                 txn_time, txn_unix = get_time_for_trans(trans_df=trans_df, client_id=client_id, is_fraud=is_fraud, time_weights=time_weights, \
#                                                         timestamps=timestamps, online=True)
#                 pos_txns.append({
#                 "client_id": client_id, "txn_time": txn_time, "unix_time":txn_unix, "amount": round(amount, 2), "type": "purchase",
#                 "channel": "POS", "category": category_name, "online": online, "merchant_id": merchant_id,
#                 "trans_loc": trans_loc, "client_ip":"not applicable",
#                 "account_to": np.nan, "is_fraud": is_fraud, "status":"approved"
#             })


            
#             #############################
#             # ГЕНЕРАЦИЯ ФРОД ТРАНЗАКЦИЙ #
#             #############################    

#             if is_fraud:
#                 # случайный выбор категории транзакции. В зависимости от частоты фрода в категории
#                 category = categories.sample(1, replace=True, weights=categories.fraud_share)
#                 category_name = category["category"].iloc[0]
#                 round_clock = category["round_clock"].iloc[0]
#                 online = category["online"].iloc[0]
#                 # средняя сумма для этой категории
#                 amt_mean = category["avg_amt"].iloc[0]
#                 # стандартное отклонение сумм для этой категории
#                 amt_std = category["amt_std"].iloc[0]
#                 # случайно сгенерированная сумма транзакции, но не менее 1
#                 # amount = max(1, np.random.normal(amt_mean, amt_std)) - amount для фрода генерится после определения правила


        
#         # Список словарей с готовыми транзакциями для одного клиента делаем геодатафреймом
#         # и присоединяем к основному геодатафрейму
#         trans_df_one_client = gpd.GeoDataFrame(pos_txns)
#         trans_df = pd.concat([trans_df, trans_df_one_client], ignore_index=True)

#     # Возвращаем целый датафрейм. Колонкой с геометрией устанавливаем trans_loc
#     return trans_df.set_geometry("trans_loc")

# Разбитая на подфункции функция `get_time_for_trans` - старая версия, до рефактора

In [ ]:
# def get_time_for_trans(trans_df, is_fraud, time_weights, timestamps, timestamps_1st_month, \
#                        online=None, rule=None, geo_distance=None, lag=None):
#     """
#     trans_df - датафрейм с транзакциями текущего клиента. Откуда брать информацию по предыдущим транзакциям клиента
#     is_fraud - boolean. Фрод или не фрод
#     time_weights - датафрейм с весами часов в периоде времени
#     timestamps - датафрейм с timestamps
#     timestamps_1st_month - сабсет timestamps отфильтрованный по первому месяцу и, если применимо, году. Чтобы генерировать первые транзакции
#     online - boolean. Онлайн или оффлайн покупка. True or False
#     rule - str. Название антифрод правила
#     geo_distance - int. Дистанция между локацией последней и текущей транзакции если фрод со сменой геолокации - в километрах
#     lag - boolean. Задержка по времени от предыдущей транзакции. Нужна для моделирования увеличения частоты транзакций.
#           Это задержка именно между последней легитимной транзакцией и серией частых фрод транзакций. Подразумевается что функция
#           get_time_for_trans будет использована в цикле, и для первой итерации lag будет True.

#     Возвращает время для генерируемой транзакции в виде pd.Timestamp и в виде unix времени
#     """
    
#     # timestamp последней транзакции клиента - может быть вынести фильтрацию по client_id из функции?
#     # Да. Вынесу фильтрацию по client_id из функции. Т.к. транзакции планируется генерировать в цикле итерируясь через клиентов.
    
#     last_txn_time = trans_df.txn_time.max()
#     last_txn_unix = trans_df.unix_time.max()
#     offline_txns = trans_df[trans_df.online == False]
#     online_txns = trans_df[trans_df.online == True]
    

#     ########################
#     # ЛЕГАЛЬНЫЕ ТРАНЗАКЦИИ #
#     ########################
    
#     # Если нет никакой предыдущей транзакции т.е. нет последнего времени совсем
#     if not is_fraud and last_txn_time is pd.NaT:
#         # берем первый месяц всего времененного периода т.к. это первая транзакция, пусть будет создана в первом месяце
#         # timestamps_1st_month = timestamps.loc[timestamps.timestamp.dt.month == timestamps.timestamp.dt.month.min()]

#         # получаем timestamp и unix время транзакции
#         txn_time, txn_unix = sample_time_for_trans(timestamps=timestamps_1st_month, time_weights=time_weights)

#         # время транзакции в виде timestamp и unix time. Здесь отдельный return, чтобы прервать дальнейшее выполнений функции
#         # т.к. это отдельный блок if-statements
#         return txn_time, txn_unix

    
#     # Если есть предыдущая транзакция

#     # берем случайный час передав веса часов
#     txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
    
#     # фильтруем по этому часу timestamp-ы и семплируем timestamp уже с равной вероятностью
#     timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
#     timestamp_sample = timestamps_subset.sample(n=1, replace=True)
        
#     # Не фрод. Текущая транзакция оффлайн и есть предыдущая оффлайн транзакция.
#     # Обеспечение минимального интервала времени между оффлайн транзакциями
#     # Это чтобы легальные транзакции не попали под фрод из-за частоты выполнения или быстроты физического перемещения клиента между транзакциями
    
#     if not is_fraud and not online and not offline_txns.empty:

#         # передаем функции сабсет из оффлайн транзакций, семплированный timestamp, онлайн ставим False, 
#         # минимально допустимая разница между другими оффлайн транзакциями - 60 минут
        
#         txn_time, txn_unix = check_min_interval_from_near_trans(txns_subset=offline_txns, timestamp_sample=timestamp_sample, online=False, \
#                                                                 offline_time_diff=60)

#     # Не фрод. Текущая транзакция онлайн и есть предыдущая онлайн транзакция.
#     # Обеспечение минимального интервала времени между онлайн транзакциями
    
#     elif not is_fraud and online and not online_txns.empty:
        
#         # передаем функции сабсет из оффлайн транзакций, семплированный timestamp, онлайн ставим True, 
#         # минимально допустимая разница между другими онлайн транзакциями - 6 минут
#         # разница от последней транзакции: от 6 до 60 минут - для случая если timestamp близко по времени к существующей транзакции
        
#         txn_time, txn_unix = check_min_interval_from_near_trans(txns_subset=offline_txns, timestamp_sample=timestamp_sample, online=False, \
#                                                                 online_time_diff=6, online_ceil=60)

#     # Случай когда есть любые предыдущие транзакции, но они другого типа: либо текущая категория онлайн, а предыдущие транзакции все оффлайн
#     # либо текущая категория оффлайн, а предыдущие транзакции все онлайн
#     elif not is_fraud:
#         # из отфильтрованного датафрейма таймстемпов семплируем один таймстемп с равной вероятностью
#         txn_time = timestamps_subset.timestamp.sample(n=1, replace=True).iloc[0]
#         txn_unix = pd_timestamp_to_unix(txn_time)
        

#     ########
#     # ФРОД #
#     ########

#     # Фрод. Правила: другая гео за короткое время либо по локации оффлайн мерчанта либо по новому ip адресу
#     elif is_fraud and rule in ["fast_geo_change", "fast_geo_change_online"]:
#         txn_time, txn_unix = generate_time_fast_geo_jump(last_txn_unix, geo_distance, threshold=800)

    
#     # Фрод. Увеличение количества транзакций в единицу времени выше установленного порога в процентах.
#     elif is_fraud and rule == "trans_freq_increase":
#         txn_time, txn_unix = gen_time_for_frequent_trans(last_txn_time, last_txn_unix, timestamps, time_weights, lag, \
#                                                          lag_interval=30, freq_low=1, freq_high=5)

#     # Фрод. Любое другое правило, где сабсет timestamp-ов семплирован ранее в соответствии с переданными весами
#     elif is_fraud:
#         # из отфильтрованного датафрейма таймстемпов семплируем один таймстемп с равной вероятностью
#         txn_time = timestamps_subset.timestamp.sample(n=1, replace=True).iloc[0]
#         txn_unix = pd_timestamp_to_unix(txn_time)


#     # время транзакции в виде timestamp и unix time
#     return txn_time, txn_unix

# Подфункция `check_min_interval_from_near_trans` - когда есть предыдущие транзакции - `Версия с багом`

In [ ]:
# def check_min_interval_from_near_trans(client_txns, timestamp_sample, online, round_clock, offline_time_diff=60, \
#                                        online_time_diff=6, online_ceil=60, general_diff=30, general_ceil=90, test=False):
#     """
#     Если для сгенерированного времени есть транзакции, которые по времени ближе заданного минимума, 
#     то создать время на основании времени последней транзакции + установленный минимальный интервал.
#     Учитывает разницу между типами транзакций: онлайн-онлайн, онлайн-оффлайн, оффлайн-оффлайн.
#     Можно поставить свои минимальные интервалы для случаев: онлайн-онлайн, онлайн-оффлайн, оффлайн-оффлайн. 
#     Для оффлайн-оффлайн - один фиксированный интервал.
#     Для онлайн-онлайн задается минимум и максимум откуда интервал берется с равной вероятностью
#     Для оффлайн-онлайн задается минимум и максимум откуда интервал берется с равной вероятностью
#     -----------------------------------------------

#     client_txns - датафрейм с транзакциями клиента.
#     timestamp_sample - случайно выбранная запись из датафрейма с таймстемпами
#     online - boolean. Онлайн или оффлайн категория
#     round_clock - boolean. Круглосуточная или дневная категория.
#     offline_time_diff - int. желаемая минимальная разница от последней оффлайн транзакции в минутах
#     online_time_diff - int. желаемая минимальная разница от последней оффлайн транзакции в минутах
#     online_ceil - int. желаемая максимальная разница от последней оффлайн транзакции в минутах
#     general_diff - int. минимально допустимая разница от последней транзакции противоположной по флагу online.
#     general_ceil - int. максимальная разница от последней транзакции противоположной по флагу online.
#                    При случае необходимости увеличения разрыва времени.
#     test - boolean. True - логировать исполнение функции в csv.
#     ------------------------------------------------
#     Возвращает pd.Timestamp и int unix время в секундах 
#     """

#     # перевод аргументов в секунды для работы с unix time
#     offline_time_diff= offline_time_diff * 60
#     online_time_diff= online_time_diff * 60
#     online_ceil= online_ceil * 60
#     general_diff = general_diff * 60
#     general_ceil = general_ceil * 60

    
#     timestamp_unix = timestamp_sample.unix_time.iloc[0]
#     # Копия, чтобы не внести изменения в исходный датафрейм
#     client_txns = client_txns.copy()
#     client_txns["abs_time_proximity"] = client_txns.unix_time.sub(timestamp_unix).abs()

#     # Запись о ближайшей по времени транзакции
#     closest_txn = client_txns.loc[client_txns.abs_time_proximity == client_txns.abs_time_proximity.min()]
#     # Онлайн или не онлайн ближайшая по времени транзакция
#     closest_online_flag = closest_txn.online.iloc[0]
#     # Разница семплированного timestamp-а с ближайшей по времени транзакцией
#     closest_time_diff = closest_txn.abs_time_proximity.iloc[0]

#     # Запись о последней транзакции
#     last_txn = client_txns.loc[client_txns.unix_time == client_txns.unix_time.max()]
#     # Онлайн или не онлайн последняя транзакция
#     last_online_flag = last_txn.online.iloc[0]
#     # unix время
#     last_txn_unix = last_txn.unix_time.iloc[0]
    
#     # Если, создаваемая транзакция оффлайн и ближайшая по времени тоже оффлайн
#     # И разница меньше допустимой
#     if not online and not closest_online_flag and closest_time_diff < offline_time_diff:
#         close_flag = "offline_to_offline"

#     # Если оффлайн транзакция и ближайшая онлайн. И разница меньше допустимой
#     elif not online and closest_online_flag and closest_time_diff < general_diff:
#         close_flag = "offline_to_online"
        
#     # Если, создаваемая транзакция онлайн и ближайшая по времени тоже онлайн
#     # И разница меньше допустимой
#     elif online and closest_online_flag and closest_time_diff < online_time_diff:
#         close_flag = "online_to_online"

#     # Если онлайн транзакция и ближайшая оффлайн. И разница меньше допустимой
#     elif online and not closest_online_flag and closest_time_diff < general_diff:
#         close_flag = "online_to_offline"
        
#     # Если нет транзакций ближе установленной разницы
#     # Просто берем изначальный timestamp
#     else:
#         close_flag = "No flag"
#         txn_unix = timestamp_unix
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Если транзакция близка по времени к другой, то согласно типам транзакций
#     # создаем другое время на основании времени и типа последней и текущей транзакции
#     if close_flag in ["offline_to_offline", "offline_to_online"]:
#         # Если последняя транзакция Онлайн. То добавляем случайную разницу для онлайн и оффлайн транзакций в установленном диапазоне
#         if last_online_flag:
#             general_random_diff = random.randint(general_diff, general_ceil)
#             txn_unix = last_txn_unix + general_random_diff
#             txn_time = pd.to_datetime(txn_unix, unit="s")
            
#         # Если последняя транзакция Оффлайн. То добавляем допустимую разницу между оффлайн транзакциями
#         elif not last_online_flag:
#             txn_unix = last_txn_unix + offline_time_diff
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Если текущая транзакция онлайн и есть онлайн/оффлайн транзакция с разницей меньше допустимой
#     elif close_flag in ["online_to_online", "online_to_offline"]:
#         # Если последняя транзакция онлайн. То добавляем случайную разницу для онлайн транзакций в установленном диапазоне
#         if last_online_flag:
#             online_random_diff = random.randint(online_time_diff, online_ceil)
#             txn_unix = last_txn_unix + online_random_diff
#             txn_time = pd.to_datetime(txn_unix, unit="s")
            
#         # Если последняя транзакция Оффлайн. То добавляем случайную разницу для онлайн и оффлайн транзакций в установленном диапазоне
#         elif not last_online_flag:
#             general_random_diff = random.randint(general_diff, general_ceil)
#             txn_unix = last_txn_unix + general_random_diff
#             txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Проверка и корректировка времени, на случай если категория дневная, и время выходит за рамки этой категории
#     # Если час меньше 8 и больше 21. Т.е. ограничение 08:00-21:59
#     if not online and not round_clock and (txn_time.hour < 8 or txn_time.hour > 21):
#         txn_time = txn_time + pd.Timedelta(10, unit="h")
#         txn_unix = pd_timestamp_to_unix(txn_time)
        
#     if not test:
#         return txn_time, txn_unix
        
#     # В тестовом режиме логируем некоторые данные в csv
#     else:
#         log_check_min_time(client_id=client_txns.client_id.iloc[0], txn_time=txn_time, txn_unix=txn_unix, online=online, closest_txn=closest_txn, \
#                            closest_online_flag=closest_online_flag, last_txn=last_txn, last_online_flag=last_online_flag, close_flag=close_flag)
#         return txn_time, txn_unix

## Подфункция генерации времени транзакции для правила `trans_freq_increase` - timestamp семплируется `первая версия с lag`

In [327]:
# def gen_time_for_frequent_trans(last_txn_time, last_txn_unix, timestamps, time_weights, lag, lag_interval=30, \
#                                 freq_low=1, freq_high=5, test=False, test_timestamp=None):
#     """
#     Функция для имитации времени нескольких частых транзакций подряд.
#     -------------------------------------------------
#     last_txn_time - pd.Timestamp последней транзакции
#     last_txn_unix - unix время последней транзакции в секундах
#     timestamps - pd.DataFrame с диапазоном pd.Timestamp-ов и их unix аналогом в секундах и колонкой часа timestamp-а
#     lag - True или False. Является ли текущая транзакция первой в серии мошеннических/подозрительных учащенных транзакций.
#           При True будет добавлен лаг равный lag_interval от предыдущей легальной транзакции
#     lag_interval - int. лаг от предыдущей транзакции в минутах
#     freq_low - int. минимальный разрыв между транзакциями в цепочке фрод транзакций, в минутах
#     freq_high - int. максимальный разрыв между транзакциями в цепочке фрод транзакций, в минутах
#     test - True или False. Тестируем мы функцию или нет. Если тестируем, то надо передать test_timestamp
#     test_timestamp - str. Дата и время в формате '1899-01-01 00:00:00'. Это замена семплируемуго внутри
#                      функции timestamp-а, который проверяется на разницу во времени с последней легальной транзакцией.
#     --------------------------------------------------
#     При test == False возвращает pd.Timestamp и unix time в секундах
#     При test == True возвращает pd.Timestamp, unix time в секундах и получившуюся разницу времени с предыдущей транзакцией в минутах в виде int
#     """

#     # перевод минут в секунды для удобства подсчета с unix time
#     lag_interval = lag_interval * 60
    
#     # если транзакция первая в серии фрод транзакций - аргумент lag равен True. И не режим тестирования функции
#     if lag and not test:
        
#         # Фильтруем timestamp-ы, оставляя те, что после последней легитимной транзакции
#         timestamp_after_last_trans = timestamps.loc[timestamps.timestamp > last_txn_time]

#         # Семплируем час первой транзакции, передав веса часов.
#         # Затем фильтруем timestamp-ы по этому часу
#         txn_hour = time_weights.hours.sample(n=1, weights=time_weights.proportion, replace=True).iloc[0]
#         timestamps_subset = timestamp_after_last_trans.loc[timestamp_after_last_trans.hour == txn_hour]
#         # семплируем таймстемп из таймстемпов по времени не ранее последней транзакции
#         timestamp_sample = timestamps_subset.sample(n=1, replace=True)
#         # Узнаем разницу между timestamp-ом и последней транзакцией
#         trans_time_diff = timestamp_sample.unix_time.sub(last_txn_unix).iloc[0]

#     # Если режим тестирования. То timestamp_sample у нас это не семпл, а переданный в аргументе timestamp
#     elif lag and test:
#         timestamp_sample = pd.DataFrame({"timestamp":pd.Series(dtype="datetime64[s]"), "unix_time":pd.Series(dtype="int64")})
#         timestamp_sample.loc[0, "timestamp"] = pd.to_datetime(test_timestamp, format="%Y-%m-%d %H:%M:%S")
#         timestamp_sample.loc[0, "unix_time"] = pd_timestamp_to_unix(timestamp_sample.timestamp.iloc[0])
#         # Узнаем разницу между timestamp-ом и последней транзакцией
#         trans_time_diff = timestamp_sample.unix_time.iloc[0] - last_txn_unix


#     # если транзакция первая в серии фрод транзакций
#     # и интервал между последней транзакцией менее 30 минут
#     # прибавить интервал 30 минут к семплированному времени текущей транзакции
#     if lag and trans_time_diff < lag_interval:
#         # print("Condition #1")
#         time_addition = lag_interval - trans_time_diff
#         txn_unix = timestamp_sample.unix_time.iloc[0] + time_addition
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # Если lag равен True, но не надо добавлять интервал
#     elif lag:
#         # print("Condition #2")
#         txn_unix = timestamp_sample.unix_time.iloc[0]
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     # для остальных случаев - когда это не первые фрод транзакции в серии
#     # просто добавляем 1-5 минут ко времени предыдущей транзакции
#     else:
#         # частота фрод транзакций. от 1 до 5 минут. Выразим в секундах для удобства расчетов
#         freq = random.randint(freq_low, freq_high) * 60
#         txn_unix = last_txn_unix + freq
#         txn_time = pd.to_datetime(txn_unix, unit="s")

#     if not test:
#         return txn_time, txn_unix
    
#     elif test and lag:
#         return txn_time, txn_unix
        
#     elif test and not lag:
#         return txn_time, txn_unix, freq

**Старые тесты `gen_time_for_frequent_trans`**

In [328]:
# weights_for_freq_trans = time_weights["Online_Fraud"]["weights"]

# Диапазон timestamp-ов для семплирования

# timestamps_freq_test = create_timestamps_range_df(start="2023-09-20", end="2023-09-21")

# # Датафрейм с последней транзакцией - перезапустить ячейку для теста ниже

# last_time = pd.to_datetime("2023-09-20 09:14:00", format="%Y-%m-%d %H:%M:%S")
# last_unix = pd_timestamp_to_unix(last_time)

# # условно основной датафрейм с транзакциями. У нас там одна легальная последняя транзакция на данный момент

# trans_test_freq_time = transactions.copy().loc[:, ['txn_time', 'unix_time','online','account_to', 'is_fraud', 'is_suspicious']]
# trans_test_freq_time.loc[0, ["txn_time","unix_time"]] = last_time, last_unix
# print(f"last_time, last_unix: {last_time, last_unix}")
# trans_test_freq_time

# # в этом скрипте нужно проверить с условиями:
# # 1. test_timestamp равен txn_time в исходном датафрейме с последней легальной транзакцией
# # 2. test_timestamp больше txn_time в исходном датафрейме с последней легальной транзакцией,на не более чем на 29 минут
# # 3. test_timestamp больше txn_time в исходном датафрейме с последней легальной транзакцией, на 30 и более минут

# trans_freq_temp_df = transactions.copy().loc[:, ['txn_time', 'unix_time','online','account_to', 'is_fraud', 'is_suspicious']]
# last_time = trans_test_freq_time.txn_time.max()
# last_unix = trans_test_freq_time.unix_time.max()

# test_mode = 3

# if test_mode == 1:
#     test_timestamp = last_time
# elif test_mode == 2:
#     test_timestamp = last_time + pd.Timedelta(7, unit="m")
# elif test_mode == 3:
#     test_timestamp = last_time + pd.Timedelta(30, unit="m")


# freq_values = []


# for i in range(1,6):
#     if i == 1:
#         lag = True

#         txn_time, txn_unix = gen_time_for_frequent_trans(last_txn_time=last_time, last_txn_unix=last_unix, timestamps=timestamps_freq_test, \
#                                                          time_weights=weights_for_freq_trans, lag=lag, \
#                                                          test=True, test_timestamp=test_timestamp)
        
#     else:
#         lag = False
    
#         txn_time, txn_unix, freq = gen_time_for_frequent_trans(last_txn_time=last_time, last_txn_unix=last_unix, timestamps=timestamps_freq_test, \
#                                                          time_weights=weights_for_freq_trans, lag=lag, \
#                                                          test=True, test_timestamp=test_timestamp)
#         freq_values.append(freq / 60)
        
#     trans_freq_temp_df.loc[i, "txn_time"] = txn_time
#     trans_freq_temp_df.loc[i, "unix_time"] = txn_unix
#     trans_freq_temp_df.loc[i, "is_fraud"] = True

    
#     last_time = txn_time
#     last_unix = txn_unix

# # средняя частота фрод транзакций - всех, кроме первой фрод транзакции
# print("\n\n Mean freq: ",pd.Series(freq_values).mean(), " minutes\n\n")

# # к последней легальной транзакции присоединяем сгенерированные частые транзакции
# trans_test_freq_time = pd.concat([trans_test_freq_time, trans_freq_temp_df])
# # разница во времени с предыдущей транзакцией, в минутах
# trans_test_freq_time["time_diff"] = trans_test_freq_time.unix_time.sub(trans_test_freq_time.unix_time.shift(1)) / 60
# trans_test_freq_time

In [329]:
# вторая запись. Первая фрод транзакция. time_diff должен быть не менее установленного lag_interval
# trans_test_freq_time.loc[1, "time_diff"]

# записи с третьей по последнюю - 2-я фрод транзакция и далее. time_diff должно быть в пределах диапазона freq
# trans_test_freq_time.loc[2:(trans_test_freq_time.shape[0] - 1)]

# txn_hour = time_weights["Offline_24h_Legit"]["weights"].hours.sample(n=1, \
#                                 weights=time_weights["Offline_24h_Legit"]["weights"].proportion, replace=True).iloc[0]
# timestamps_subset = timestamps.loc[timestamps.hour == txn_hour]
# timestamp_sample = timestamps_subset.sample(n=1, replace=True)

# last_time = pd.to_datetime("2024-07-24 19:26:00", format="%Y-%m-%d %H:%M:%S")
# last_unix = pd_timestamp_to_unix(last_time)
# last_time, last_unix